In [10]:
import mne
from mne.decoding import CSP
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc, confusion_matrix
import os
from mne import Epochs, pick_types, __main__
from mne import Epochs, create_info, events_from_annotations
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler, label_binarize
import joblib
from pyriemann.classification import MDM, TSclassifier
from pyriemann.estimation import Covariances
le = LabelEncoder()
import pickle
import plotly.graph_objects as go
import pandas as pd
import glob
from mrmr import mrmr_classif
import mrmr

import pandas as pd
random_state=42
mne.set_log_level('CRITICAL')
#WEIRD = Westernized, Educated, Industrialized, Rich, and Democratic


In [11]:
colors = {
    'AW': '#ef8a62',
    'MO': '#67a9cf',
    'MI': '#999999',
    'AW_2': '#613828'
}

#data_folder = "raw_data"
data_folder = "cleaned_dataset"
ERP_channels =  ['Fz', 'C3', 'Cz', 'C4', 'Pz', 'Oz'] # Updated channel list
FBCSP_channels = ['C3', 'Cz', 'C4', 'PO7', 'Pz', 'PO8'] # Updated channel list

In [12]:
def AW_annotations_online(raw, AW_labels):
    #new_annotations = raw.annotations.description
    new_annotations=[]
    for annotation in raw.annotations:
        if '%' not in annotation['description']:
            new_annotations.append(annotation)
    for j in new_annotations:
        if j['description']=='Start_cue_Arm':
            filename = 'conditions_male.xlsx'
            break
        if j['description']=='Start_cue_Leg':
            filename = 'conditions_female.xlsx'
            break   
    df_condition = pd.read_excel(filename)
    n=0
    for j in new_annotations:
        if j['description'] =='Start_cue_Arm' or j['description']=='Start_cue_Leg':
            if df_condition.loc[n, 'Cue'] == 'AW':
                if AW_labels == 'New_labels':
                    if df_condition.loc[n, 'Visual_Cue']=='APLAUDIR':
                        j['description']='AW_APLAUDIR'
                    elif df_condition.loc[n, 'Visual_Cue']=='PATEAR':
                        j['description']='AW_PATEAR'
                    elif df_condition.loc[n,'Condition']=='Leg':
                        j['description']='AW_Leg'
                    elif df_condition.loc[n,'Condition']=='Arm':
                        j['description']='AW_Arm'
                else:
                    j['description']='_'.join(df_condition.values[n][1:-1])
            else:
                j['description']='_'.join(df_condition.values[n][1:-1])
            n=n+1
    
    annotations = mne.Annotations(
        onset=[a['onset'] for a in new_annotations],
        duration=[a['duration'] for a in new_annotations],
        description=[a['description'] for a in new_annotations]
    )
    raw.set_annotations(annotations)     

In [13]:
def create_offline_epochs(raw, events, event_id, channels, tmin_tmax, baseline, clean_data=True):
    
    if clean_data:
        raw.filter(l_freq=1.0, h_freq=None,  verbose=False)
        raw.filter(l_freq=None, h_freq=40.0,  verbose=False)
    picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False)
    epochs = Epochs(
            raw,
            events,
            event_id,
            tmin=tmin_tmax[0],
            tmax=tmin_tmax[1],
            proj=False,
            picks=picks,
            baseline=baseline,
            preload=True,
         verbose=False)
    
    return epochs.pick_channels(channels) 

In [14]:
def create_online_epochs(filename, condition, aw_labels, channels, tmin_tmax, baseline, clean_data = True):
   
   raw = mne.io.read_raw_eeglab(filename, preload=True)
   #
   if clean_data:
      raw.filter(l_freq=1.0, h_freq=None,  verbose=False)
      raw.filter(l_freq=None, h_freq=40.0,  verbose=False)
      
   picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False)
    #Start_cue_leg_TRETEN  Start_cue_Arm_KLATSCHEN
   AW_annotations_online(raw, aw_labels)
   events, events_id = events_from_annotations(raw, verbose=False)

   if condition == 'AW':
      events, events_id = events_from_annotations(raw, verbose=False)
      if aw_labels == 'New_labels':
         event_id= {'AW_Arm': events_id['AW_Arm'], 'AW_Leg': events_id['AW_Leg'], 
                   'AW_APLAUDIR': events_id['AW_APLAUDIR'], 'AW_PATEAR': events_id['AW_PATEAR']}
      else:
         event_id= {'AW_Arm': events_id['AW_Arm'], 'AW_Leg': events_id['AW_Leg']}
   elif condition == 'MI':
      event_id={'MI_Arm': events_id['MI_Arm'], 'MI_Leg': events_id['MI_Leg']}
   elif condition == 'MO':
      event_id={'MO_Arm': events_id['MO_Arm'], 'MO_Leg': events_id['MO_Leg']}

        
   epochs = Epochs(
    raw,
    events,
    event_id=event_id,
    tmin=tmin_tmax[0],
    tmax=tmin_tmax[1],
    proj=True,
    picks=picks,
    baseline=baseline,
    preload=True,
    detrend=0)

   return epochs.pick_channels(channels)   

In [15]:
def select_discriminative_band(X, y, channels, sfreq, f_range=(5, 35), threshold=0.05):
    from scipy.signal import welch
    from scipy.stats import pearsonr
    """
    Selects a discriminative frequency band from EEG data.

    Args:
        X (numpy.ndarray): EEG data of shape (n_trials, n_channels, n_samples).
        y (numpy.ndarray): Trial labels of shape (n_trials,).
        channels (list): List of channel names.
        sfreq (int): Sampling frequency of the EEG data.
        f_range (tuple): Frequency range to consider (default: 5-35 Hz).
        threshold (float): Threshold for band expansion (default: 0.05).

    Returns:
        tuple: Selected frequency band [f0, f1].
    """
    n_trials, n_channels, n_samples = X.shape
    f_start, f_end = f_range
    
    scores = {}
    for f in range(f_start, f_end + 1):
        scores[f] = np.zeros(n_channels)
        for c_idx, c in enumerate(channels):
            band_power = []
            for i in range(n_trials):
                # Compute power spectral density using Welch's method
                f_welch, psd = welch(X[i, c_idx, :], sfreq, window='hamming', nfft=n_samples)
                # Find the power at frequency f
                f_idx = np.argmin(np.abs(f_welch - f))
                band_power.append(np.log(psd[f_idx]))
            
            # Calculate correlation between band power and labels
            correlation, _ = pearsonr(band_power, y)
            scores[f][c_idx] = correlation if not np.isnan(correlation) else 0.0  # Handle NaN

    # Find the channel with the maximum correlation for each frequency
    f_max_channels = {f: channels[np.argmax(np.abs(scores[f]))] for f in scores}

    # Find the frequency with the maximum absolute correlation across all channels
    f_max = max(scores, key=lambda f: np.max(np.abs(scores[f])))
    f_score_max = np.max(np.abs(scores[f_max]))

    # Initialize frequency band
    f0 = f_max
    f1 = f_max

    # Expand lower bound
    while f0 - 1 >= f_start and np.max(np.abs(scores[f0 - 1])) >= f_score_max * (1 - threshold):
        f0 -= 1

    # Expand upper bound
    while f1 + 1 <= f_end and np.max(np.abs(scores[f1 + 1])) >= f_score_max * (1 - threshold):
        f1 += 1

    return f0-3, f1+3

In [16]:
def perform_mrmr_feature_selection(features, labels, num_features=10):
    # Convert the scaled features to a pandas DataFrame
    df = pd.DataFrame(features)

    # Add target variable to the DataFrame
    df['target'] = labels

    # Convert target to integer type (required by mRMR)
    df['target'] = df['target'].astype(int)

    # Perform MRMR feature selection
    # Note: The features argument here is already scaled by the calling function
    selected_features = mrmr_classif(df.drop(columns='target'), df['target'], num_features, show_progress=False)

    # Create datasets with selected features
    X_selected = features[:, selected_features]
    return X_selected, selected_features


In [17]:
def extract_fbcsp_features_for_cv(epochs, filtered_bands=None):
    if filtered_bands is None:
        filtered_bands = [(8, 12), (12, 16), (16, 20), (20, 24), (24, 28), (13, 30)]
    filtered_epochs_list = []
    for l_freq, h_freq in filtered_bands:
        epochs_filtered = epochs.copy().filter(l_freq=l_freq, h_freq=h_freq,
                                               fir_design='firwin', verbose=False)
        filtered_epochs_list.append(epochs_filtered)
    return filtered_epochs_list

In [18]:
def extract_fbcsp_features_and_model(epochs, labels, random_state=42):
    clf = SVC(kernel='linear', random_state=random_state)

    #clf = RandomForestClassifier(n_estimators=100, random_state=random_state, n_jobs=-1)
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)
    accuracies = []

    filtered_epochs_list = extract_fbcsp_features_for_cv(epochs)

    for train_index, test_index in skf.split(epochs, labels):
        csp_features_list_train = []
        csp_features_list_test = []
        n_components = 4
        csp = CSP(n_components=n_components, reg='ledoit_wolf', log=True, norm_trace=False)
        y_train = labels[train_index]
        y_test = labels[test_index]

        for epochs_filtered in filtered_epochs_list:
            epochs_data = epochs_filtered.get_data(copy=False) * 1e6
            X_train_fold = epochs_data[train_index]
            X_test_fold = epochs_data[test_index]

            csp.fit(X_train_fold, y_train)
            csp_features_train = csp.transform(X_train_fold)
            csp_features_test = csp.transform(X_test_fold)

            csp_features_list_train.append(csp_features_train)
            csp_features_list_test.append(csp_features_test)

        fbcsp_features_train = np.concatenate(csp_features_list_train, axis=1)
        fbcsp_features_test = np.concatenate(csp_features_list_test, axis=1)
        
        # Instantiate and fit scaler for this fold
        scaler = StandardScaler()
        fbcsp_features_train_scaled = scaler.fit_transform(fbcsp_features_train)
        fbcsp_features_test_scaled = scaler.transform(fbcsp_features_test)
        
        mrmr_fbcsp_features_train, selected_features = perform_mrmr_feature_selection(fbcsp_features_train_scaled, y_train, num_features=10)
        mrmr_fbcsp_features_test = fbcsp_features_test_scaled[:, selected_features]

        clf.fit(mrmr_fbcsp_features_train, y_train)
        y_pred = clf.predict(mrmr_fbcsp_features_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

    final_csp_features_list = []
    final_csp = CSP(n_components=4, reg='ledoit_wolf', log=True, norm_trace=False)
    for epochs_filtered in filtered_epochs_list:
        epochs_data = epochs_filtered.get_data(copy=False) * 1e6
        final_csp.fit(epochs_data, labels)
        final_csp_features_list.append(final_csp.transform(epochs_data))
    final_fbcsp_features = np.concatenate(final_csp_features_list, axis=1)

    final_scaler = StandardScaler()
    final_fbcsp_features_scaled = final_scaler.fit_transform(final_fbcsp_features)

    mrmr_final_fbcsp_features, selected_features = perform_mrmr_feature_selection(final_fbcsp_features_scaled, labels, num_features=10)
    
    final_clf = SVC(kernel='linear', random_state=random_state)
    #final_clf = RandomForestClassifier(n_estimators=100, random_state=random_state, n_jobs=-1)
    final_clf.fit(mrmr_final_fbcsp_features, labels)
    
    return {'average_accuracy': np.mean(accuracies), 'selected_features': selected_features}, final_clf, final_csp, final_scaler, mrmr_final_fbcsp_features

In [19]:
def offline_combined_classification(fbcsp_features, erp_features, labels):
    """
    Performs classification on a combined feature set (FBCSP and ERP).
    """
    clf = SVC(kernel='linear', random_state=random_state)

    #clf = RandomForestClassifier(n_estimators=100, random_state=random_state, n_jobs=-1)
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)
    accuracies = []

    combined_features = np.concatenate([fbcsp_features, erp_features], axis=1)
    
    for train_index, test_index in skf.split(combined_features, labels):
        X_train, X_test = combined_features[train_index], combined_features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    final_clf = SVC(kernel='linear', random_state=random_state)

    #final_clf = RandomForestClassifier(n_estimators=100, random_state=random_state, n_jobs=-1)
    final_clf.fit(combined_features, labels)
    
    return {'average_accuracy': np.mean(accuracies)}, final_clf, combined_features


In [20]:
def extract_erp_features_and_model(epochs, labels, random_state=42):
    clf = SVC(kernel='linear', random_state=random_state)
    #clf = RandomForestClassifier(n_estimators=100, random_state=random_state, n_jobs=-1)
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)
    accuracies = []

    window_duration_s = 0.5
    step_size_s = 0.5
    sfreq = epochs.info['sfreq']
    window_samples = int(window_duration_s * sfreq)
    step_samples = int(step_size_s * sfreq)
    start_time_s = 0.0
    end_time_s = epochs.tmax
    start_time_sample = epochs.time_as_index(start_time_s)[0]
    end_time_sample = epochs.time_as_index(end_time_s)[0]

    all_erp_features = []
    for epoch_data in epochs.get_data():
        trial_features = []
        for start_idx in np.arange(start_time_sample, end_time_sample - window_samples + 1, step_samples):
            end_idx = start_idx + window_samples
            window_data = epoch_data[:, start_idx:end_idx]
            window_features = np.mean(window_data, axis=1)
            trial_features.append(window_features)
        all_erp_features.append(np.concatenate(trial_features))
    final_erp_features = np.array(all_erp_features)

    for train_index, test_index in skf.split(final_erp_features, labels):
        X_train, X_test = final_erp_features[train_index], final_erp_features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        mrmr_train_features, selected_features = perform_mrmr_feature_selection(X_train_scaled, y_train, num_features=10)
        mrmr_test_features = X_test_scaled[:, selected_features]

        clf.fit(mrmr_train_features, y_train)
        y_pred = clf.predict(mrmr_test_features)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

    final_scaler = StandardScaler()
    final_erp_features_scaled = final_scaler.fit_transform(final_erp_features)
    mrmr_final_erp_features, selected_features = perform_mrmr_feature_selection(final_erp_features_scaled, labels, num_features=10)
    final_erp_features_selected = final_erp_features_scaled[:, selected_features]

    #final_clf = RandomForestClassifier(n_estimators=100, random_state=random_state, n_jobs=-1)
    final_clf = SVC(kernel='linear', random_state=random_state)

    final_clf.fit(final_erp_features_selected, labels)

    return {'average_accuracy': np.mean(accuracies), 'selected_features': selected_features}, final_clf, final_scaler, final_erp_features_selected

In [21]:
def align_epochs_and_labels(fbcsp_epochs, erp_epochs, arm_label='Start_cue_Arm', leg_label='Start_cue_Leg'):

    fbcsp_events = fbcsp_epochs.events
    erp_events = erp_epochs.events

            # Find common events
    common_event_ids_fbcsp = set(tuple(row) for row in fbcsp_events)
    common_event_ids_erp = set(tuple(row) for row in erp_events)
    common_events = list(common_event_ids_fbcsp.intersection(common_event_ids_erp))

            # Get indices for alignment
    fbcsp_indices = [np.where((fbcsp_events == common_event).all(axis=1))[0][0] for common_event in common_events]
    erp_indices = [np.where((erp_events == common_event).all(axis=1))[0][0] for common_event in common_events]

            # Align epochs
    fbcsp_aligned = fbcsp_epochs[fbcsp_indices]
    erp_aligned = erp_epochs[erp_indices]

            # Create label mapping and labels
    mapping_offline = {
    fbcsp_aligned[arm_label].events[:, -1][0]: 0,
    fbcsp_aligned[leg_label].events[:, -1][0]: 1}
    labels_offline = np.vectorize(mapping_offline.get)(fbcsp_aligned.events[:, -1])
    return fbcsp_aligned, erp_aligned, labels_offline

In [22]:
epochs_AW_FBCSP=[]
epochs_MI_FBCSP=[]
epochs_MO_FBCSP=[]
epochs_AW_ERP=[]
epochs_MI_ERP=[]
epochs_MO_ERP =[]
subjects = [str(i) for i in range(1,30)]

for subject in subjects:
    set_files = glob.glob(os.path.join(data_folder, subject, '*.set'))
    for file in set_files:
        filename = file
        raw = mne.io.read_raw_eeglab(filename, preload=True, verbose=False)  # Load data
        event_id = {'Start_cue_Arm': 3, 'Start_cue_Leg': 4}
        events, _ = events_from_annotations(raw, event_id, verbose=False)
        if 'AW' in file:
            epochs_AW_FBCSP.append(create_offline_epochs(raw, events, event_id, FBCSP_channels, [0.5, 3.0], None, clean_data=False))
            epochs_AW_ERP.append(create_offline_epochs(raw, events, event_id, ERP_channels, [-0.5, 2.5], [-0.5, 0], clean_data=False))
        elif 'MI' in file:
            epochs_MI_FBCSP.append(create_offline_epochs(raw, events, event_id, FBCSP_channels, [0.5, 3.0], None, clean_data=False))
            epochs_MI_ERP.append(create_offline_epochs(raw, events, event_id, ERP_channels, [-0.5, 2.5], [-0.5, 0], clean_data=False))
        elif 'MO' in file:
            epochs_MO_FBCSP.append(create_offline_epochs(raw, events, event_id, FBCSP_channels, [0.5, 3.0], None, clean_data=False))
            epochs_MO_ERP.append(create_offline_epochs(raw, events, event_id, ERP_channels, [-0.5, 2.5], [-0.5, 0], clean_data=False))

In [23]:

AW_results = {'FBCSP': {}, 'ERP': {}, 'Combined': {}}
    
for n, subject in enumerate(subjects):
    AW_FBCSP = epochs_AW_FBCSP[n]
    AW_ERP = epochs_AW_ERP[n]
        
    AW_FBCSP_aligned, AW_ERP_aligned, labels_offline = align_epochs_and_labels(AW_FBCSP, AW_ERP)
        
    print(f"--- Processing Subject {subject} ---")

        # 1. FBCSP-only Classification
    fbcsp_results, fbcsp_clf, fbcsp_csp, fbcsp_scaler, fbcsp_features = extract_fbcsp_features_and_model(AW_FBCSP_aligned, labels_offline)
    AW_results['FBCSP'][subject] = fbcsp_results
    joblib.dump(fbcsp_clf, os.path.join('classifiers', f'AW_FBCSP_clf_subject_{n}.pkl'))
    joblib.dump(fbcsp_csp, os.path.join('features_extractors', f'AW_FBCSP_csp_subject_{n}.pkl'))
    joblib.dump(fbcsp_scaler, os.path.join('features_extractors', f'AW_FBCSP_csp_scaler_subject_{n}.pkl'))

    print(f"FBCSP-only accuracy: {fbcsp_results['average_accuracy']:.4f}")

    # 2. ERP-only Classification
    erp_results, erp_clf, erp_scaler, erp_features = extract_erp_features_and_model(AW_ERP_aligned, labels_offline)
    AW_results['ERP'][subject] = erp_results
    joblib.dump(erp_clf, os.path.join('classifiers', f'AW_ERP_clf_subject_{n}.pkl'))
    joblib.dump(erp_scaler, os.path.join('features_extractors', f'AW_ERP_scaler_subject_{n}.pkl'))

    print(f"ERP-only accuracy: {erp_results['average_accuracy']:.4f}")

        # 3. FBCSP + ERP Combined Classification
    combined_results, combined_clf, combined_features = offline_combined_classification(fbcsp_features, erp_features, labels_offline)
    AW_results['Combined'][subject] = combined_results
    joblib.dump(combined_clf, os.path.join('classifiers', f'AW_Combined_clf_subject_{n}.pkl'))
    print(f"Combined accuracy: {combined_results['average_accuracy']:.4f}")



--- Processing Subject 1 ---
FBCSP-only accuracy: 0.3533
ERP-only accuracy: 0.4967
Combined accuracy: 0.6267
--- Processing Subject 2 ---
FBCSP-only accuracy: 0.4929
ERP-only accuracy: 0.4071
Combined accuracy: 0.6589
--- Processing Subject 3 ---
FBCSP-only accuracy: 0.4786
ERP-only accuracy: 0.4679
Combined accuracy: 0.6732
--- Processing Subject 4 ---
FBCSP-only accuracy: 0.5661
ERP-only accuracy: 0.4696
Combined accuracy: 0.7304
--- Processing Subject 5 ---
FBCSP-only accuracy: 0.5196
ERP-only accuracy: 0.5750
Combined accuracy: 0.6696
--- Processing Subject 6 ---
FBCSP-only accuracy: 0.3250
ERP-only accuracy: 0.5250
Combined accuracy: 0.7000
--- Processing Subject 7 ---
FBCSP-only accuracy: 0.4375
ERP-only accuracy: 0.4625
Combined accuracy: 0.6196
--- Processing Subject 8 ---
FBCSP-only accuracy: 0.5857
ERP-only accuracy: 0.5286
Combined accuracy: 0.7179
--- Processing Subject 9 ---
FBCSP-only accuracy: 0.4643
ERP-only accuracy: 0.4946
Combined accuracy: 0.5946
--- Processing Subj

In [24]:
MI_results = {'FBCSP': {}, 'ERP': {}, 'Combined': {}}
    
for n, subject in enumerate(subjects):
    MI_FBCSP = epochs_MI_FBCSP[n]
    MI_ERP = epochs_MI_ERP[n]
        
    MI_FBCSP_aligned, MI_ERP_aligned, labels_offline = align_epochs_and_labels(MI_FBCSP, MI_ERP)
        
    print(f"--- Processing Subject {subject} ---")

    # 1. FBCSP-only Classification
    fbcsp_results, fbcsp_clf, fbcsp_csp, fbcsp_scaler, fbcsp_features = extract_fbcsp_features_and_model(MI_FBCSP_aligned, labels_offline)
    MI_results['FBCSP'][subject] = fbcsp_results
    joblib.dump(fbcsp_clf, os.path.join('classifiers', f'MI_FBCSP_clf_subject_{n}.pkl'))
    joblib.dump(fbcsp_csp, os.path.join('features_extractors', f'MI_FBCSP_csp_subject_{n}.pkl'))
    joblib.dump(fbcsp_scaler, os.path.join('features_extractors', f'MI_FBCSP_csp_scaler_subject_{n}.pkl'))

    print(f"FBCSP-only accuracy: {fbcsp_results['average_accuracy']:.4f}")

    # 2. ERP-only Classification
    erp_results, erp_clf, erp_scaler, erp_features = extract_erp_features_and_model(MI_ERP_aligned, labels_offline)
    MI_results['ERP'][subject] = erp_results
    joblib.dump(erp_clf, os.path.join('classifiers', f'MI_ERP_clf_subject_{n}.pkl'))
    joblib.dump(erp_scaler, os.path.join('features_extractors', f'MI_ERP_scaler_subject_{n}.pkl'))

    print(f"ERP-only accuracy: {erp_results['average_accuracy']:.4f}")

    # 3. FBCSP + ERP Combined Classification
    combined_results, combined_clf, combined_features = offline_combined_classification(fbcsp_features, erp_features, labels_offline)
    MI_results['Combined'][subject] = combined_results
    joblib.dump(combined_clf, os.path.join('classifiers', f'MI_Combined_clf_subject_{n}.pkl'))
    print(f"Combined accuracy: {combined_results['average_accuracy']:.4f}")


--- Processing Subject 1 ---
FBCSP-only accuracy: 0.6036
ERP-only accuracy: 0.6089
Combined accuracy: 0.7268
--- Processing Subject 2 ---
FBCSP-only accuracy: 0.4714
ERP-only accuracy: 0.3911
Combined accuracy: 0.5839
--- Processing Subject 3 ---
FBCSP-only accuracy: 0.5071
ERP-only accuracy: 0.5571
Combined accuracy: 0.6196
--- Processing Subject 4 ---
FBCSP-only accuracy: 0.6125
ERP-only accuracy: 0.4625
Combined accuracy: 0.6875
--- Processing Subject 5 ---
FBCSP-only accuracy: 0.5500
ERP-only accuracy: 0.5000
Combined accuracy: 0.6250
--- Processing Subject 6 ---
FBCSP-only accuracy: 0.4696
ERP-only accuracy: 0.4161
Combined accuracy: 0.5071
--- Processing Subject 7 ---
FBCSP-only accuracy: 0.5750
ERP-only accuracy: 0.5250
Combined accuracy: 0.5750
--- Processing Subject 8 ---
FBCSP-only accuracy: 0.5411
ERP-only accuracy: 0.4179
Combined accuracy: 0.6375
--- Processing Subject 9 ---
FBCSP-only accuracy: 0.4625
ERP-only accuracy: 0.5054
Combined accuracy: 0.6304
--- Processing Subj

In [25]:
MO_results = {'FBCSP': {}, 'ERP': {}, 'Combined': {}}
    
for n, subject in enumerate(subjects):
    MO_FBCSP = epochs_MO_FBCSP[n]
    MO_ERP = epochs_MO_ERP[n]
        
    MO_FBCSP_aligned, MO_ERP_aligned, labels_offline = align_epochs_and_labels(MO_FBCSP, MO_ERP)
        
    print(f"--- Processing Subject {subject} ---")

    # 1. FBCSP-only Classification
    fbcsp_results, fbcsp_clf, fbcsp_csp, fbcsp_scaler, fbcsp_features = extract_fbcsp_features_and_model(MO_FBCSP_aligned, labels_offline)
    MO_results['FBCSP'][subject] = fbcsp_results
    joblib.dump(fbcsp_clf, os.path.join('classifiers', f'MO_FBCSP_clf_subject_{n}.pkl'))
    joblib.dump(fbcsp_csp, os.path.join('features_extractors', f'MO_FBCSP_csp_subject_{n}.pkl'))
    joblib.dump(fbcsp_scaler, os.path.join('features_extractors', f'MO_FBCSP_csp_scaler_subject_{n}.pkl'))

    print(f"FBCSP-only accuracy: {fbcsp_results['average_accuracy']:.4f}")

    # 2. ERP-only Classification
    erp_results, erp_clf, erp_scaler, erp_features = extract_erp_features_and_model(MO_ERP_aligned, labels_offline)
    MO_results['ERP'][subject] = erp_results
    joblib.dump(erp_clf, os.path.join('classifiers', f'MO_ERP_clf_subject_{n}.pkl'))
    joblib.dump(erp_scaler, os.path.join('features_extractors', f'MO_ERP_scaler_subject_{n}.pkl'))

    print(f"ERP-only accuracy: {erp_results['average_accuracy']:.4f}")

    # 3. FBCSP + ERP Combined Classification
    combined_results, combined_clf, combined_features = offline_combined_classification(fbcsp_features, erp_features, labels_offline)
    MO_results['Combined'][subject] = combined_results
    joblib.dump(combined_clf, os.path.join('classifiers', f'MO_Combined_clf_subject_{n}.pkl'))
    print(f"Combined accuracy: {combined_results['average_accuracy']:.4f}")


--- Processing Subject 1 ---
FBCSP-only accuracy: 0.5250
ERP-only accuracy: 0.6018
Combined accuracy: 0.8054
--- Processing Subject 2 ---
FBCSP-only accuracy: 0.6375
ERP-only accuracy: 0.4375
Combined accuracy: 0.6500
--- Processing Subject 3 ---
FBCSP-only accuracy: 0.4304
ERP-only accuracy: 0.4643
Combined accuracy: 0.5821
--- Processing Subject 4 ---
FBCSP-only accuracy: 0.6125
ERP-only accuracy: 0.5500
Combined accuracy: 0.6000
--- Processing Subject 5 ---
FBCSP-only accuracy: 0.4821
ERP-only accuracy: 0.5607
Combined accuracy: 0.6714
--- Processing Subject 6 ---
FBCSP-only accuracy: 0.4518
ERP-only accuracy: 0.5821
Combined accuracy: 0.6339
--- Processing Subject 7 ---
FBCSP-only accuracy: 0.5321
ERP-only accuracy: 0.4661
Combined accuracy: 0.6375
--- Processing Subject 8 ---
FBCSP-only accuracy: 0.4786
ERP-only accuracy: 0.4310
Combined accuracy: 0.7000
--- Processing Subject 9 ---
FBCSP-only accuracy: 0.4946
ERP-only accuracy: 0.4054
Combined accuracy: 0.4696
--- Processing Subj

In [26]:
for subject in subjects:
    print(subject, AW_results['ERP'][subject]['average_accuracy'], MI_results['ERP'][subject]['average_accuracy'], MO_results['ERP'][subject]['average_accuracy'])


1 0.4966666666666666 0.6089285714285715 0.6017857142857144
2 0.40714285714285714 0.39107142857142857 0.4375
3 0.46785714285714286 0.5571428571428572 0.4642857142857143
4 0.4696428571428572 0.4625 0.55
5 0.575 0.5 0.5607142857142857
6 0.525 0.41607142857142854 0.5821428571428572
7 0.4624999999999999 0.525 0.4660714285714286
8 0.5285714285714286 0.4178571428571428 0.4309523809523809
9 0.4946428571428571 0.5053571428571428 0.40535714285714286
10 0.605 0.6133333333333333 0.4666666666666667
11 0.6333333333333333 0.42857142857142855 0.49523809523809514
12 0.56 0.5 0.5625
13 0.5285714285714286 0.49107142857142855 0.5166666666666667
14 0.7125 0.6733333333333333 0.31785714285714284
15 0.5446428571428571 0.40476190476190477 0.48571428571428565
16 0.4428571428571428 0.5966666666666667 0.45892857142857146
17 0.5410714285714285 0.4738095238095238 0.4999999999999999
18 0.5499999999999999 0.4428571428571429 0.4767857142857143
19 0.5303571428571427 0.575 0.4125
20 0.5267857142857142 0.55 0.4375
21 0.5

In [27]:
for subject in subjects:
    print(subject, AW_results['FBCSP'][subject]['average_accuracy'], MI_results['FBCSP'][subject]['average_accuracy'], MO_results['FBCSP'][subject]['average_accuracy'])

1 0.35333333333333333 0.6035714285714284 0.525
2 0.4928571428571429 0.4714285714285714 0.6375
3 0.47857142857142854 0.5071428571428571 0.4303571428571429
4 0.5660714285714286 0.6125 0.6125
5 0.5196428571428572 0.55 0.4821428571428572
6 0.325 0.46964285714285714 0.45178571428571423
7 0.4375 0.575 0.5321428571428571
8 0.5857142857142856 0.5410714285714285 0.47857142857142854
9 0.4642857142857143 0.4625 0.4946428571428571
10 0.43000000000000005 0.4933333333333333 0.5533333333333333
11 0.37333333333333335 0.4999999999999999 0.45
12 0.48 0.55 0.5125
13 0.5571428571428572 0.4928571428571429 0.4666666666666666
14 0.45 0.47333333333333333 0.4196428571428572
15 0.5053571428571428 0.5047619047619047 0.5285714285714285
16 0.45714285714285713 0.4966666666666666 0.5232142857142856
17 0.45357142857142857 0.5190476190476191 0.4333333333333333
18 0.46964285714285714 0.5857142857142856 0.46428571428571425
19 0.5696428571428571 0.5125 0.48571428571428577
20 0.3267857142857143 0.6 0.4375
21 0.54464285714

In [28]:
for subject in subjects:
    print(subject, AW_results['Combined'][subject]['average_accuracy'], MI_results['Combined'][subject]['average_accuracy'], MO_results['Combined'][subject]['average_accuracy'])

1 0.6266666666666667 0.7267857142857143 0.8053571428571429
2 0.6589285714285714 0.5839285714285715 0.65
3 0.6732142857142857 0.6196428571428572 0.5821428571428571
4 0.7303571428571428 0.6875 0.6
5 0.6696428571428571 0.625 0.6714285714285715
6 0.7 0.5071428571428571 0.6339285714285714
7 0.6196428571428572 0.575 0.6375
8 0.7178571428571429 0.6375 0.7000000000000001
9 0.5946428571428571 0.6303571428571428 0.46964285714285714
10 0.525 0.6633333333333333 0.6199999999999999
11 0.6699999999999999 0.7714285714285714 0.5833333333333333
12 0.7200000000000001 0.5125 0.65
13 0.6428571428571428 0.625 0.5928571428571427
14 0.6625 0.6566666666666666 0.49821428571428567
15 0.5214285714285716 0.580952380952381 0.5428571428571428
16 0.5660714285714287 0.7533333333333333 0.5892857142857142
17 0.5767857142857142 0.43571428571428567 0.7
18 0.6839285714285714 0.5999999999999999 0.5732142857142858
19 0.6392857142857142 0.6625 0.4482142857142857
20 0.6964285714285714 0.775 0.7
21 0.5839285714285715 0.51333333

In [29]:
# Create a DataFrame to organize the data
data = []
for subject_id, scores in AW_results['Combined'].items():
    data.append({
        'Subject': subject_id,
        'Condition': 'AW',
        'Accuracy': scores['average_accuracy']
    })
for subject_id, scores in MO_results['Combined'].items():
    data.append({
        'Subject': subject_id,
        'Condition': 'MO',
        'Accuracy': scores['average_accuracy']
    })
for subject_id, scores in MI_results['Combined'].items():
    data.append({
        'Subject': subject_id,
        'Condition': 'MI',
        'Accuracy': scores['average_accuracy']
    })

df = pd.DataFrame(data)
df_grouped = df.iloc[:,1:].groupby('Condition').agg(['mean', 'std'])
print(df_grouped)

# Calculate means and stds for error bars
means = df.groupby('Condition')['Accuracy'].mean()
stds = df.groupby('Condition')['Accuracy'].std()

# Create Plotly grouped bar chart with error bars
fig = go.Figure()

for condition in df['Condition'].unique():
    cond_df = df[df['Condition'] == condition]
    fig.add_trace(go.Bar(
        x=cond_df['Subject'],
        y=cond_df['Accuracy'],
        name=condition,
        marker_color=colors[condition],
        error_y=dict(
            type='data',
            array=[stds[condition]] * len(cond_df),  # same std for all bars in this condition
            visible=True
        )
    ))

# Add chance level line
fig.add_hline(y=0.5, line_width=1, line_color="gray")

# Update layout to show subject IDs on x-axis ticks
fig.update_layout(
    barmode='group',
    xaxis_title='Subject',
    yaxis_title='Accuracy',
    title='Accuracy per Subject and Condition  - Combined FBCSP and ERP',
    legend_title_text='Condition',
    xaxis=dict(
        tickmode='array',
        tickvals=df['Subject'].unique(),
        ticktext=df['Subject'].unique()
    ),
    yaxis=dict(range=[0.3, 1.0], tick0=0.4, dtick=0.1),
)

fig.show()


           Accuracy          
               mean       std
Condition                    
AW         0.636823  0.064405
MI         0.625837  0.083702
MO         0.632492  0.082547


In [30]:
# Create a DataFrame to organize the data
data = []
for subject_id, scores in AW_results['FBCSP'].items():
    data.append({
        'Subject': subject_id,
        'Condition': 'AW',
        'Accuracy': scores['average_accuracy']
    })
for subject_id, scores in MO_results['FBCSP'].items():
    data.append({
        'Subject': subject_id,
        'Condition': 'MO',
        'Accuracy': scores['average_accuracy']
    })
for subject_id, scores in MI_results['FBCSP'].items():
    data.append({
        'Subject': subject_id,
        'Condition': 'MI',
        'Accuracy': scores['average_accuracy']
    })

df = pd.DataFrame(data)
df_grouped = df.iloc[:,1:].groupby('Condition').agg(['mean', 'std'])
print(df_grouped)

# Calculate means and stds for error bars
means = df.groupby('Condition')['Accuracy'].mean()
stds = df.groupby('Condition')['Accuracy'].std()

# Create Plotly grouped bar chart with error bars
fig = go.Figure()

for condition in df['Condition'].unique():
    cond_df = df[df['Condition'] == condition]
    fig.add_trace(go.Bar(
        x=cond_df['Subject'],
        y=cond_df['Accuracy'],
        name=condition,
        marker_color=colors[condition],
        error_y=dict(
            type='data',
            array=[stds[condition]] * len(cond_df),  # same std for all bars in this condition
            visible=True
        )
    ))

# Add chance level line
fig.add_hline(y=0.5, line_width=1, line_color="gray")

# Update layout to show subject IDs on x-axis ticks
fig.update_layout(
    barmode='group',
    xaxis_title='Subject',
    yaxis_title='Accuracy',
    title='Accuracy per Subject and Condition  -  FBCSP',
    legend_title_text='Condition',
    xaxis=dict(
        tickmode='array',
        tickvals=df['Subject'].unique(),
        ticktext=df['Subject'].unique()
    ),
    yaxis=dict(range=[0.0, 1.0], tick0=0.4, dtick=0.1),
)

fig.show()


           Accuracy          
               mean       std
Condition                    
AW         0.483896  0.080044
MI         0.523149  0.045985
MO         0.504056  0.067074


In [31]:
# Create a DataFrame to organize the data
data = []
for subject_id, scores in AW_results['ERP'].items():
    data.append({
        'Subject': subject_id,
        'Condition': 'AW',
        'Accuracy': scores['average_accuracy']
    })
for subject_id, scores in MO_results['ERP'].items():
    data.append({
        'Subject': subject_id,
        'Condition': 'MO',
        'Accuracy': scores['average_accuracy']
    })
for subject_id, scores in MI_results['ERP'].items():
    data.append({
        'Subject': subject_id,
        'Condition': 'MI',
        'Accuracy': scores['average_accuracy']
    })

df = pd.DataFrame(data)
df_grouped = df.iloc[:,1:].groupby('Condition').agg(['mean', 'std'])
print(df_grouped)

# Calculate means and stds for error bars
means = df.groupby('Condition')['Accuracy'].mean()
stds = df.groupby('Condition')['Accuracy'].std()

# Create Plotly grouped bar chart with error bars
fig = go.Figure()

for condition in df['Condition'].unique():
    cond_df = df[df['Condition'] == condition]
    fig.add_trace(go.Bar(
        x=cond_df['Subject'],
        y=cond_df['Accuracy'],
        name=condition,
        marker_color=colors[condition],
        error_y=dict(
            type='data',
            array=[stds[condition]] * len(cond_df),  # same std for all bars in this condition
            visible=True
        )
    ))

# Add chance level line
fig.add_hline(y=0.5, line_width=1, line_color="gray")

# Update layout to show subject IDs on x-axis ticks
fig.update_layout(
    barmode='group',
    xaxis_title='Subject',
    yaxis_title='Accuracy',
    title='Accuracy per Subject and Condition  - ERP',
    legend_title_text='Condition',
    xaxis=dict(
        tickmode='array',
        tickvals=df['Subject'].unique(),
        ticktext=df['Subject'].unique()
    ),
    yaxis=dict(range=[0.3, 1.0], tick0=0.4, dtick=0.1),
)

fig.show()


           Accuracy          
               mean       std
Condition                    
AW         0.520505  0.084276
MI         0.504269  0.069334
MO         0.502968  0.071038


In [32]:
def online_overtime_classification_from_whole_epochs(epochs_fbcsp, epochs_erp, labels,
                                   models_and_selectors):
    """
    Performs classification over time using a sliding window on unseen data.

    Args:
        epochs_fbcsp (mne.Epochs): FBCSP epochs (aligned and preprocessed).
        epochs_erp (mne.Epochs): ERP epochs (aligned and preprocessed).
        labels (np.ndarray): The true labels for the epochs.
        models_and_selectors (dict): A dictionary containing all pre-trained models
                                     and selected feature indices.

    Returns:
        dict: A dictionary with accuracies for each classification method over time.
    """

    # Load pre-trained models and selectors from the dictionary
    fbcsp_clf = models_and_selectors['fbcsp_clf']
    fbcsp_csp = models_and_selectors['fbcsp_csp']
    fbcsp_selected_features = models_and_selectors['fbcsp_selected_features']
    fbcsp_scaler = models_and_selectors['fbcsp_scaler']
    erp_clf = models_and_selectors['erp_clf']
    erp_selected_features = models_and_selectors['erp_selected_features']
    erp_scaler = models_and_selectors['erp_scaler']
    combined_clf = models_and_selectors['combined_clf']

    all_fbcsp_preds = []
    all_erp_preds = []
    all_combined_preds = []

    # Define online window parameters. This is the main sliding window.
    window_duration_s = 2.5
    step_size_s = 0.25

    sfreq = epochs_fbcsp.info['sfreq']
    
    window_samples = int(window_duration_s * sfreq)
    step_samples = int(step_size_s * sfreq)

    # Determine the time range for the sliding window
    # The online epoch for FBCSP should go up to 5s, so we slide the 2.5s window over this period.
    start_time_online = 0.0
    end_time_online = 5.0
    
    try:
        start_sample_idx = epochs_fbcsp.time_as_index(start_time_online)[0]
        end_sample_idx_loop = epochs_fbcsp.time_as_index(end_time_online)[0]
    except IndexError:
        print("Warning: The online epochs do not contain the full time range for the sliding window. "
              "Please create the online epochs with a wider time window.")
        return None

    start_times = np.arange(start_sample_idx, end_sample_idx_loop - window_samples + 1, step_samples)
    
    # List to store the center time of each window
    window_labels = []

    for start_sample in start_times:
        end_sample = start_sample + window_samples

        # --- FBCSP Feature Extraction for the 2.5s window ---
        # The FBCSP offline training used a [0.5, 3.0] window, which has a duration of 2.5s.
        # We need to crop to that window. The start time of the online window is `epochs_fbcsp.times[start_sample]`.
        # So we crop from that start time to a duration of 2.5s.
        fbcsp_window_epochs = epochs_fbcsp.copy().crop(tmin=epochs_fbcsp.times[start_sample],
                                                       tmax=epochs_fbcsp.times[start_sample + window_samples - 1],
                                                       include_tmax=True)
        
        fbcsp_features_list = []
        filtered_bands = [(8, 12), (12, 16), (16, 20), (20, 24), (24, 28), (13, 30)]
        
        for l_freq, h_freq in filtered_bands:
            temp_filtered_epochs = fbcsp_window_epochs.copy().filter(l_freq, h_freq, verbose=False)
            epochs_data = temp_filtered_epochs.get_data(copy=False) * 1e6
            csp_features = fbcsp_csp.transform(epochs_data)
            fbcsp_features_list.append(csp_features)
        
        fbcsp_features = np.concatenate(fbcsp_features_list, axis=1)
        
        # Apply pre-trained scaler before mRMR feature selection
        fbcsp_features = fbcsp_scaler.transform(fbcsp_features)
        
        # Apply mRMR feature selection with the pre-trained indices
        fbcsp_features_mrmr = fbcsp_features[:, fbcsp_selected_features]

        # --- ERP Feature Extraction for the 2.5s window with nested sliding windows ---
        # The ERP offline training used a [-0.5, 2.5] window. We need to crop the online data
        # to a 2.5s window starting from the same relative point.
        erp_window_epochs = epochs_erp.copy().crop(tmin=epochs_erp.times[start_sample],
                                                   tmax=epochs_erp.times[start_sample + window_samples - 1],
                                                   include_tmax=True)

        # Define nested ERP window parameters from your offline code
        erp_window_duration_s = 0.5
        erp_step_size_s = 0.5
        erp_window_samples = int(erp_window_duration_s * sfreq)
        erp_step_samples = int(erp_step_size_s * sfreq)

        all_erp_features_window = []
        for trial_data in erp_window_epochs.get_data():
            trial_features = []
            for start_idx in np.arange(0, trial_data.shape[-1] - erp_window_samples + 1, erp_step_samples):
                end_idx = start_idx + erp_window_samples
                window_data = trial_data[:, start_idx:end_idx]
                window_features = np.mean(window_data, axis=1)
                trial_features.append(window_features)
            
            all_erp_features_window.append(np.concatenate(trial_features))

        erp_features = np.array(all_erp_features_window)
        
        # Apply pre-trained scaler before mRMR feature selection
        erp_features = erp_scaler.transform(erp_features)
        
        # Apply mRMR feature selection with the pre-trained indices
        erp_features_mrmr = erp_features[:, erp_selected_features]

        # --- Classification for all epochs in the current window ---
        fbcsp_preds = fbcsp_clf.predict(fbcsp_features_mrmr)
        erp_preds = erp_clf.predict(erp_features_mrmr)
        
        combined_features = np.concatenate([fbcsp_features_mrmr, erp_features_mrmr], axis=1)
        combined_preds = combined_clf.predict(combined_features)
        
        all_fbcsp_preds.append(fbcsp_preds)
        all_erp_preds.append(erp_preds)
        all_combined_preds.append(combined_preds)
        
        window_labels.append(epochs_fbcsp.times[start_sample + window_samples // 2])

    # Calculate the accuracy for each time window
    fbcsp_accuracies = [accuracy_score(labels, preds) for preds in all_fbcsp_preds]
    erp_accuracies = [accuracy_score(labels, preds) for preds in all_erp_preds]
    combined_accuracies = [accuracy_score(labels, preds) for preds in all_combined_preds]

    return {
        'times': window_labels,
        'fbcsp_accuracies': fbcsp_accuracies,
        'erp_accuracies': erp_accuracies,
        'combined_accuracies': combined_accuracies
    }


In [33]:

all_online_AW_results = {}
for n, subject in enumerate(subjects):
    online_file = os.path.join(data_folder, str(subject), f'{subject}_cleaned_online.set')
    if os.path.exists(online_file):
        print(f"Online file found: {online_file}")
    else:
        print(f"Online file not found: {online_file}")
        continue
    all_online_AW_results[subject] = {}

    AW_FBCSP_csp= joblib.load(os.path.join('features_extractors', 'AW_FBCSP_csp_subject_'+str(n)+'.pkl'))
    AW_FBCSP_scaler = joblib.load(os.path.join('features_extractors', 'AW_FBCSP_csp_scaler_subject_'+str(n)+'.pkl'))
    AW_FBCSP_clf = joblib.load(os.path.join('classifiers', 'AW_FBCSP_clf_subject_'+str(n)+'.pkl'))
    AW_ERP_clf = joblib.load(os.path.join('classifiers', 'AW_ERP_clf_subject_'+str(n)+'.pkl'))
    AW_ERP_scaler = joblib.load(os.path.join('features_extractors', 'AW_ERP_scaler_subject_'+str(n)+'.pkl'))
    AW_combined_clf = joblib.load(os.path.join('classifiers', 'AW_Combined_clf_subject_'+str(n)+'.pkl'))
    epochs_online_AW_FBCSP = create_online_epochs(online_file, 'AW', None, FBCSP_channels, [-0.5, 5.0], None, clean_data=False)
    epochs_online_AW_ERP = create_online_epochs(online_file, 'AW', None, ERP_channels, [-0.5, 5.0], [-0.5, 0], clean_data=False)

    AW_FBCSP_online_aligned, AW_ERP_online_aligned, labels_online_AW = align_epochs_and_labels(epochs_online_AW_FBCSP, epochs_online_AW_ERP,'AW_Arm', 'AW_Leg')

    models_and_selectors = {'fbcsp_clf':AW_FBCSP_clf,'fbcsp_csp': AW_FBCSP_csp, 'fbcsp_selected_features': AW_results['FBCSP'][subject]['selected_features'], 'fbcsp_scaler': AW_FBCSP_scaler,
                            'erp_clf': AW_ERP_clf, 'erp_selected_features': AW_results['ERP'][subject]['selected_features'], 'erp_scaler': AW_ERP_scaler,
                            'combined_clf': AW_combined_clf}
    
    results = online_overtime_classification_from_whole_epochs(AW_FBCSP_online_aligned, AW_ERP_online_aligned, labels_online_AW, models_and_selectors) 
    all_online_AW_results[subject] = results

Online file found: cleaned_dataset\1\1_cleaned_online.set
Online file not found: cleaned_dataset\2\2_cleaned_online.set
Online file found: cleaned_dataset\3\3_cleaned_online.set
Online file found: cleaned_dataset\4\4_cleaned_online.set
Online file found: cleaned_dataset\5\5_cleaned_online.set
Online file found: cleaned_dataset\6\6_cleaned_online.set
Online file found: cleaned_dataset\7\7_cleaned_online.set
Online file found: cleaned_dataset\8\8_cleaned_online.set
Online file found: cleaned_dataset\9\9_cleaned_online.set
Online file found: cleaned_dataset\10\10_cleaned_online.set
Online file found: cleaned_dataset\11\11_cleaned_online.set
Online file found: cleaned_dataset\12\12_cleaned_online.set
Online file found: cleaned_dataset\13\13_cleaned_online.set
Online file found: cleaned_dataset\14\14_cleaned_online.set
Online file found: cleaned_dataset\15\15_cleaned_online.set
Online file found: cleaned_dataset\16\16_cleaned_online.set
Online file found: cleaned_dataset\17\17_cleaned_onlin

In [34]:
all_online_MI_results = {}
for n, subject in enumerate(subjects):
    online_file = os.path.join(data_folder, str(subject), f'{subject}_cleaned_online.set')
    if os.path.exists(online_file):
        print(f"Online file found: {online_file}")
    else:
        print(f"Online file not found: {online_file}")
        continue
    all_online_MI_results[subject] = {}

    MI_FBCSP_csp = joblib.load(os.path.join('features_extractors', f'MI_FBCSP_csp_subject_{n}.pkl'))
    MI_FBCSP_scaler = joblib.load(os.path.join('features_extractors', f'MI_FBCSP_csp_scaler_subject_{n}.pkl'))
    MI_FBCSP_clf = joblib.load(os.path.join('classifiers', f'MI_FBCSP_clf_subject_{n}.pkl'))
    MI_ERP_clf = joblib.load(os.path.join('classifiers', f'MI_ERP_clf_subject_{n}.pkl'))
    MI_ERP_scaler = joblib.load(os.path.join('features_extractors', f'MI_ERP_scaler_subject_{n}.pkl'))
    MI_combined_clf = joblib.load(os.path.join('classifiers', f'MI_Combined_clf_subject_{n}.pkl'))
    epochs_online_MI_FBCSP = create_online_epochs(online_file, 'MI', None, FBCSP_channels, [-0.5, 5.0], None, clean_data=False)
    epochs_online_MI_ERP = create_online_epochs(online_file, 'MI', None, ERP_channels, [-0.5, 5.0], [-0.5, 0], clean_data=False)

    MI_FBCSP_online_aligned, MI_ERP_online_aligned, labels_online_MI = align_epochs_and_labels(
        epochs_online_MI_FBCSP, epochs_online_MI_ERP, 'MI_Arm', 'MI_Leg'
    )

    models_and_selectors = {
        'fbcsp_clf': MI_FBCSP_clf,
        'fbcsp_csp': MI_FBCSP_csp,
        'fbcsp_selected_features': MI_results['FBCSP'][subject]['selected_features'],
        'fbcsp_scaler': MI_FBCSP_scaler,
        'erp_clf': MI_ERP_clf,
        'erp_selected_features': MI_results['ERP'][subject]['selected_features'],
        'erp_scaler': MI_ERP_scaler,
        'combined_clf': MI_combined_clf
    }

    results = online_overtime_classification_from_whole_epochs(
        MI_FBCSP_online_aligned, MI_ERP_online_aligned, labels_online_MI, models_and_selectors
    )
    all_online_MI_results[subject] = results

Online file found: cleaned_dataset\1\1_cleaned_online.set
Online file not found: cleaned_dataset\2\2_cleaned_online.set
Online file found: cleaned_dataset\3\3_cleaned_online.set
Online file found: cleaned_dataset\4\4_cleaned_online.set
Online file found: cleaned_dataset\5\5_cleaned_online.set
Online file found: cleaned_dataset\6\6_cleaned_online.set
Online file found: cleaned_dataset\7\7_cleaned_online.set
Online file found: cleaned_dataset\8\8_cleaned_online.set
Online file found: cleaned_dataset\9\9_cleaned_online.set
Online file found: cleaned_dataset\10\10_cleaned_online.set
Online file found: cleaned_dataset\11\11_cleaned_online.set
Online file found: cleaned_dataset\12\12_cleaned_online.set
Online file found: cleaned_dataset\13\13_cleaned_online.set
Online file found: cleaned_dataset\14\14_cleaned_online.set
Online file found: cleaned_dataset\15\15_cleaned_online.set
Online file found: cleaned_dataset\16\16_cleaned_online.set
Online file found: cleaned_dataset\17\17_cleaned_onlin

In [35]:
all_online_MO_results = {}
for n, subject in enumerate(subjects):
    online_file = os.path.join(data_folder, str(subject), f'{subject}_cleaned_online.set')
    if os.path.exists(online_file):
        print(f"Online file found: {online_file}")
    else:
        print(f"Online file not found: {online_file}")
        continue
    all_online_MO_results[subject] = {}

    MO_FBCSP_csp = joblib.load(os.path.join('features_extractors', f'MO_FBCSP_csp_subject_{n}.pkl'))
    MO_FBCSP_scaler = joblib.load(os.path.join('features_extractors', f'MO_FBCSP_csp_scaler_subject_{n}.pkl'))
    MO_FBCSP_clf = joblib.load(os.path.join('classifiers', f'MO_FBCSP_clf_subject_{n}.pkl'))
    MO_ERP_clf = joblib.load(os.path.join('classifiers', f'MO_ERP_clf_subject_{n}.pkl'))
    MO_ERP_scaler = joblib.load(os.path.join('features_extractors', f'MO_ERP_scaler_subject_{n}.pkl'))
    MO_combined_clf = joblib.load(os.path.join('classifiers', f'MO_Combined_clf_subject_{n}.pkl'))
    epochs_online_MO_FBCSP = create_online_epochs(online_file, 'MO', None, FBCSP_channels, [-0.5, 5.0], None, clean_data=False)
    epochs_online_MO_ERP = create_online_epochs(online_file, 'MO', None, ERP_channels, [-0.5, 5.0], [-0.5, 0], clean_data=False)

    MO_FBCSP_online_aligned, MO_ERP_online_aligned, labels_online_MO = align_epochs_and_labels(
        epochs_online_MO_FBCSP, epochs_online_MO_ERP, 'MO_Arm', 'MO_Leg'
    )

    models_and_selectors = {
        'fbcsp_clf': MO_FBCSP_clf,
        'fbcsp_csp': MO_FBCSP_csp,
        'fbcsp_selected_features': MO_results['FBCSP'][subject]['selected_features'],
        'fbcsp_scaler': MO_FBCSP_scaler,
        'erp_clf': MO_ERP_clf,
        'erp_selected_features': MO_results['ERP'][subject]['selected_features'],
        'erp_scaler': MO_ERP_scaler,
        'combined_clf': MO_combined_clf
    }

    results = online_overtime_classification_from_whole_epochs(
        MO_FBCSP_online_aligned, MO_ERP_online_aligned, labels_online_MO, models_and_selectors
    )
    all_online_MO_results[subject] = results

Online file found: cleaned_dataset\1\1_cleaned_online.set
Online file not found: cleaned_dataset\2\2_cleaned_online.set
Online file found: cleaned_dataset\3\3_cleaned_online.set
Online file found: cleaned_dataset\4\4_cleaned_online.set
Online file found: cleaned_dataset\5\5_cleaned_online.set
Online file found: cleaned_dataset\6\6_cleaned_online.set
Online file found: cleaned_dataset\7\7_cleaned_online.set
Online file found: cleaned_dataset\8\8_cleaned_online.set
Online file found: cleaned_dataset\9\9_cleaned_online.set
Online file found: cleaned_dataset\10\10_cleaned_online.set
Online file found: cleaned_dataset\11\11_cleaned_online.set
Online file found: cleaned_dataset\12\12_cleaned_online.set
Online file found: cleaned_dataset\13\13_cleaned_online.set
Online file found: cleaned_dataset\14\14_cleaned_online.set
Online file found: cleaned_dataset\15\15_cleaned_online.set
Online file found: cleaned_dataset\16\16_cleaned_online.set
Online file found: cleaned_dataset\17\17_cleaned_onlin

In [36]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import warnings

# Suppress the UserWarning from Plotly
warnings.filterwarnings('ignore', category=UserWarning, module='plotly.graph_objects')

# Prepare data for plotting
plot_data = {}
for subject, results in all_online_AW_results.items():
    if not results:
        continue
    
    # Calculate mean accuracy for each model across all time points
    fbcsp_mean = np.mean(results['fbcsp_accuracies'])
    erp_mean = np.mean(results['erp_accuracies'])
    combined_mean = np.mean(results['combined_accuracies'])
    
    # Find the best model based on mean accuracy
    means = {'FBCSP': fbcsp_mean, 'ERP': erp_mean, 'Combined': combined_mean}
    best_model_name = max(means, key=means.get)
    
    # Store the accuracies of the best model and the model name
    plot_data[subject] = {
        'model_name': best_model_name,
        'accuracies': results[f'{best_model_name.lower()}_accuracies']
    }

# Create a DataFrame for easy plotting
x_times = all_online_AW_results['1']['times']
y_subjects = []
z_accuracies = []
custom_data = []

for subject, data in plot_data.items():
    y_subjects.append(f"Subject {subject}: {data['model_name']}")
    z_accuracies.append(data['accuracies'])

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    x=x_times,
    y=y_subjects,
    z=z_accuracies,
    colorscale=[[0.0, 'red'], [0.5, 'white'], [1.0, 'blue']],
    zmin=0,
    zmax=1,
    colorbar=dict(title="Accuracy")
))

# Set layout titles and labels
fig.update_layout(
    height=800,
    width=1200,
    title='Best Model Accuracy Over Time for Each Subject',
    xaxis_title='Time (s)',
    yaxis_title='Subject and Best Model',
    yaxis_autorange='reversed',
    xaxis_tickmode='linear',
    xaxis_dtick=0.5
)

fig.show()

In [37]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import warnings

# Suppress the UserWarning from Plotly
warnings.filterwarnings('ignore', category=UserWarning, module='plotly.graph_objects')

# Prepare data for plotting
plot_data = {}
for subject, results in all_online_MI_results.items():
    if not results:
        continue
    
    # Calculate mean accuracy for each model across all time points
    fbcsp_mean = np.mean(results['fbcsp_accuracies'])
    erp_mean = np.mean(results['erp_accuracies'])
    combined_mean = np.mean(results['combined_accuracies'])
    
    # Find the best model based on mean accuracy
    means = {'FBCSP': fbcsp_mean, 'ERP': erp_mean, 'Combined': combined_mean}
    best_model_name = max(means, key=means.get)
    
    # Store the accuracies of the best model and the model name
    plot_data[subject] = {
        'model_name': best_model_name,
        'accuracies': results[f'{best_model_name.lower()}_accuracies']
    }

# Create a DataFrame for easy plotting
x_times = all_online_MI_results['1']['times']
y_subjects = []
z_accuracies = []
custom_data = []

for subject, data in plot_data.items():
    y_subjects.append(f"Subject {subject}: {data['model_name']}")
    z_accuracies.append(data['accuracies'])

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    x=x_times,
    y=y_subjects,
    z=z_accuracies,
    colorscale=[[0.0, 'red'], [0.5, 'white'], [1.0, 'blue']],
    zmin=0,
    zmax=1,
    colorbar=dict(title="Accuracy")
))

# Set layout titles and labels
fig.update_layout(
    height=800,
    width=1200,
    title='Best Model Accuracy Over Time for Each Subject',
    xaxis_title='Time (s)',
    yaxis_title='Subject and Best Model',
    yaxis_autorange='reversed',
    xaxis_tickmode='linear',
    xaxis_dtick=0.5
)

fig.show()

In [38]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import warnings

# Suppress the UserWarning from Plotly
warnings.filterwarnings('ignore', category=UserWarning, module='plotly.graph_objects')

# Prepare data for plotting
plot_data = {}
for subject, results in all_online_MO_results.items():
    if not results:
        continue
    
    # Calculate mean accuracy for each model across all time points
    fbcsp_mean = np.mean(results['fbcsp_accuracies'])
    erp_mean = np.mean(results['erp_accuracies'])
    combined_mean = np.mean(results['combined_accuracies'])
    
    # Find the best model based on mean accuracy
    means = {'FBCSP': fbcsp_mean, 'ERP': erp_mean, 'Combined': combined_mean}
    best_model_name = max(means, key=means.get)
    
    # Store the accuracies of the best model and the model name
    plot_data[subject] = {
        'model_name': best_model_name,
        'accuracies': results[f'{best_model_name.lower()}_accuracies']
    }

# Create a DataFrame for easy plotting
x_times = all_online_MO_results['1']['times']
y_subjects = []
z_accuracies = []
custom_data = []

for subject, data in plot_data.items():
    y_subjects.append(f"Subject {subject}: {data['model_name']}")
    z_accuracies.append(data['accuracies'])

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    x=x_times,
    y=y_subjects,
    z=z_accuracies,
    colorscale=[[0.0, 'red'], [0.5, 'white'], [1.0, 'blue']],
    zmin=0,
    zmax=1,
    colorbar=dict(title="Accuracy")
))

# Set layout titles and labels
fig.update_layout(
    height=800,
    width=1200,
    title='Best Model Accuracy Over Time for Each Subject',
    xaxis_title='Time (s)',
    yaxis_title='Subject and Best Model',
    yaxis_autorange='reversed',
    xaxis_tickmode='linear',
    xaxis_dtick=0.5
)

fig.show()

In [39]:
def online_whole_data_classification(epochs_fbcsp, epochs_erp, labels,
                                     fbcsp_clf, fbcsp_csp, erp_clf, combined_clf,
                                     fbcsp_selected_features, erp_selected_features,
                                     fbcsp_scaler, erp_scaler):
    """
    Performs classification on the entire, unseen epochs and returns the accuracy.

    Args:
        epochs_fbcsp (mne.Epochs): FBCSP epochs (aligned and preprocessed).
        epochs_erp (mne.Epochs): ERP epochs (aligned and preprocessed).
        labels (np.ndarray): The true labels for the epochs.
        fbcsp_clf, fbcsp_csp, erp_clf, combined_clf: Pre-trained models.
        fbcsp_selected_features (np.ndarray): mRMR indices for FBCSP.
        erp_selected_features (np.ndarray): mRMR indices for ERP.
        fbcsp_scaler (StandardScaler): Pre-trained scaler for FBCSP features.
        erp_scaler (StandardScaler): Pre-trained scaler for ERP features.

    Returns:
        dict: A dictionary with the overall accuracy for each classification method.
    """

    # --- FBCSP Feature Extraction on the whole epoch ---
    fbcsp_features_list = []
    # FBCSP bands from your offline code
    filtered_bands = [(8, 12), (12, 16), (16, 20), (20, 24), (24, 28), (13, 30)]

    for l_freq, h_freq in filtered_bands:
        temp_filtered_epochs = epochs_fbcsp.copy().filter(l_freq, h_freq, verbose=False)
        epochs_data = temp_filtered_epochs.get_data(copy=False) * 1e6
        csp_features = fbcsp_csp.transform(epochs_data)
        fbcsp_features_list.append(csp_features)

    final_fbcsp_features = np.concatenate(fbcsp_features_list, axis=1)

    # Apply pre-trained scaler before mRMR feature selection
    final_fbcsp_features = fbcsp_scaler.transform(final_fbcsp_features)

    # Apply mRMR feature selection with the pre-trained indices
    final_fbcsp_features = final_fbcsp_features[:, fbcsp_selected_features]

    # --- ERP Feature Extraction on the whole epoch with sliding window ---
    # The time window for online whole data classification must match offline training
    window_duration_s = 0.5
    step_size_s = 0.5
    sfreq = epochs_erp.info['sfreq']

    window_samples = int(window_duration_s * sfreq)
    step_samples = int(step_size_s * sfreq)

    # Define the time range for the sliding window to match offline training.
    start_time_s = 0.0
    end_time_s = 2.5
    
    # Correctly crop the online epochs to match the offline training window
    epochs_erp_cropped = epochs_erp.copy().crop(tmin=start_time_s, tmax=end_time_s, include_tmax=False)

    all_erp_features = []
    for epoch_data in epochs_erp_cropped.get_data():
        trial_features = []
        for start_idx in np.arange(0, epoch_data.shape[-1] - window_samples + 1, step_samples):
            end_idx = start_idx + window_samples
            window_data = epoch_data[:, start_idx:end_idx]
            window_features = np.mean(window_data, axis=1)
            trial_features.append(window_features)

        all_erp_features.append(np.concatenate(trial_features))

    final_erp_features = np.array(all_erp_features)

    # Apply pre-trained scaler before mRMR feature selection
    final_erp_features = erp_scaler.transform(final_erp_features)

    # Apply mRMR feature selection with the pre-trained indices
    final_erp_features = final_erp_features[:, erp_selected_features]

    # --- Classification and Accuracy Calculation ---

    fbcsp_preds = fbcsp_clf.predict(final_fbcsp_features)
    fbcsp_accuracy = accuracy_score(labels, fbcsp_preds)

    erp_preds = erp_clf.predict(final_erp_features)
    erp_accuracy = accuracy_score(labels, erp_preds)

    combined_features = np.concatenate([final_fbcsp_features, final_erp_features], axis=1)
    combined_preds = combined_clf.predict(combined_features)
    combined_accuracy = accuracy_score(labels, combined_preds)

    return {
        'fbcsp_accuracy': fbcsp_accuracy,
        'erp_accuracy': erp_accuracy,
        'combined_accuracy': combined_accuracy
    }

In [40]:
whole_epochs_results_AW = {}
for n, subject in enumerate(subjects):
    online_file = os.path.join(data_folder, str(subject), f'{subject}_cleaned_online.set')
    
    if os.path.exists(online_file):
        print(f"Online file found: {online_file}")
    else:
        print(f"Online file not found: {online_file}")
        continue
    whole_epochs_results_AW[subject] = {}

    # Load pre-trained models
    fbcsp_clf = joblib.load(os.path.join('classifiers', f'AW_FBCSP_clf_subject_{n}.pkl'))
    fbcsp_csp = joblib.load(os.path.join('features_extractors', f'AW_FBCSP_csp_subject_{n}.pkl'))
    fbcsp_scaler = joblib.load(os.path.join('features_extractors', f'AW_FBCSP_csp_scaler_subject_{n}.pkl'))
    erp_clf = joblib.load(os.path.join('classifiers', f'AW_ERP_clf_subject_{n}.pkl'))
    erp_scaler = joblib.load(os.path.join('features_extractors', f'AW_ERP_scaler_subject_{n}.pkl'))
    combined_clf = joblib.load(os.path.join('classifiers', f'AW_Combined_clf_subject_{n}.pkl'))

    # Create aligned epochs (this part is the same as before)
    epochs_online_AW_FBCSP = create_online_epochs(online_file, 'AW', None, FBCSP_channels, [-0.5, 5.0], None, clean_data=False)
    epochs_online_AW_ERP = create_online_epochs(online_file, 'AW', None, ERP_channels, [-0.5, 5.0], [-0.5, 0], clean_data=False)

    AW_FBCSP_online_aligned, AW_ERP_online_aligned, labels_online_AW = align_epochs_and_labels(epochs_online_AW_FBCSP, epochs_online_AW_ERP,'AW_Arm', 'AW_Leg')

    # Run the new whole-data classification method
    whole_data_results = online_whole_data_classification(AW_FBCSP_online_aligned, AW_ERP_online_aligned, labels_online_AW,
                                                          fbcsp_clf, fbcsp_csp, erp_clf, combined_clf, 
                                                          AW_results['FBCSP'][subject]['selected_features'], AW_results['ERP'][subject]['selected_features'],
                                                          fbcsp_scaler, erp_scaler)
    whole_epochs_results_AW[subject] = whole_data_results
    print(f"Whole-data classification results for Subject {subject}:")
    print(whole_data_results)

Online file found: cleaned_dataset\1\1_cleaned_online.set
Whole-data classification results for Subject 1:
{'fbcsp_accuracy': 0.5, 'erp_accuracy': 0.65, 'combined_accuracy': 0.5}
Online file not found: cleaned_dataset\2\2_cleaned_online.set
Online file found: cleaned_dataset\3\3_cleaned_online.set
Whole-data classification results for Subject 3:
{'fbcsp_accuracy': 0.5, 'erp_accuracy': 0.5, 'combined_accuracy': 0.5}
Online file found: cleaned_dataset\4\4_cleaned_online.set
Whole-data classification results for Subject 4:
{'fbcsp_accuracy': 0.5, 'erp_accuracy': 0.45, 'combined_accuracy': 0.5}
Online file found: cleaned_dataset\5\5_cleaned_online.set
Whole-data classification results for Subject 5:
{'fbcsp_accuracy': 0.5, 'erp_accuracy': 0.3888888888888889, 'combined_accuracy': 0.5}
Online file found: cleaned_dataset\6\6_cleaned_online.set
Whole-data classification results for Subject 6:
{'fbcsp_accuracy': 0.631578947368421, 'erp_accuracy': 0.3684210526315789, 'combined_accuracy': 0.47368

In [41]:
whole_epochs_results_MI = {}
for n, subject in enumerate(subjects):
    online_file = os.path.join(data_folder, str(subject), f'{subject}_cleaned_online.set')
    
    if os.path.exists(online_file):
        print(f"Online file found: {online_file}")
    else:
        print(f"Online file not found: {online_file}")
        continue
    whole_epochs_results_MI[subject] = {}

    # Load pre-trained models
    fbcsp_clf = joblib.load(os.path.join('classifiers', f'MI_FBCSP_clf_subject_{n}.pkl'))
    fbcsp_csp = joblib.load(os.path.join('features_extractors', f'MI_FBCSP_csp_subject_{n}.pkl'))
    fbcsp_scaler = joblib.load(os.path.join('features_extractors', f'MI_FBCSP_csp_scaler_subject_{n}.pkl'))
    erp_clf = joblib.load(os.path.join('classifiers', f'MI_ERP_clf_subject_{n}.pkl'))
    erp_scaler = joblib.load(os.path.join('features_extractors', f'MI_ERP_scaler_subject_{n}.pkl'))
    combined_clf = joblib.load(os.path.join('classifiers', f'MI_Combined_clf_subject_{n}.pkl'))

    # Create aligned epochs
    epochs_online_MI_FBCSP = create_online_epochs(online_file, 'MI', None, FBCSP_channels, [-0.5, 5.0], None, clean_data=False)
    epochs_online_MI_ERP = create_online_epochs(online_file, 'MI', None, ERP_channels, [-0.5, 5.0], [-0.5, 0], clean_data=False)

    MI_FBCSP_online_aligned, MI_ERP_online_aligned, labels_online_MI = align_epochs_and_labels(
        epochs_online_MI_FBCSP, epochs_online_MI_ERP, 'MI_Arm', 'MI_Leg'
    )

    # Run the whole-data classification method
    whole_data_results = online_whole_data_classification(
        MI_FBCSP_online_aligned, MI_ERP_online_aligned, labels_online_MI,
        fbcsp_clf, fbcsp_csp, erp_clf, combined_clf, MI_results['FBCSP'][subject]['selected_features'],
        MI_results['ERP'][subject]['selected_features'],
        fbcsp_scaler, erp_scaler
    )
    whole_epochs_results_MI[subject] = whole_data_results
    print(f"Whole-data classification results for Subject {subject}:")
    print(whole_data_results)

Online file found: cleaned_dataset\1\1_cleaned_online.set
Whole-data classification results for Subject 1:
{'fbcsp_accuracy': 0.47368421052631576, 'erp_accuracy': 0.3157894736842105, 'combined_accuracy': 0.47368421052631576}
Online file not found: cleaned_dataset\2\2_cleaned_online.set
Online file found: cleaned_dataset\3\3_cleaned_online.set
Whole-data classification results for Subject 3:
{'fbcsp_accuracy': 0.5263157894736842, 'erp_accuracy': 0.42105263157894735, 'combined_accuracy': 0.5263157894736842}
Online file found: cleaned_dataset\4\4_cleaned_online.set
Whole-data classification results for Subject 4:
{'fbcsp_accuracy': 0.5263157894736842, 'erp_accuracy': 0.5789473684210527, 'combined_accuracy': 0.5263157894736842}
Online file found: cleaned_dataset\5\5_cleaned_online.set
Whole-data classification results for Subject 5:
{'fbcsp_accuracy': 0.6470588235294118, 'erp_accuracy': 0.4117647058823529, 'combined_accuracy': 0.47058823529411764}
Online file found: cleaned_dataset\6\6_cle

In [42]:
whole_epochs_results_MO = {}
for n, subject in enumerate(subjects):
    online_file = os.path.join(data_folder, str(subject), f'{subject}_cleaned_online.set')
    
    if os.path.exists(online_file):
        print(f"Online file found: {online_file}")
    else:
        print(f"Online file not found: {online_file}")
        continue
    whole_epochs_results_MO[subject] = {}

    # Load pre-trained models
    fbcsp_clf = joblib.load(os.path.join('classifiers', f'MO_FBCSP_clf_subject_{n}.pkl'))
    fbcsp_csp = joblib.load(os.path.join('features_extractors', f'MO_FBCSP_csp_subject_{n}.pkl'))
    fbcsp_scaler = joblib.load(os.path.join('features_extractors', f'MO_FBCSP_csp_scaler_subject_{n}.pkl'))
    erp_clf = joblib.load(os.path.join('classifiers', f'MO_ERP_clf_subject_{n}.pkl'))
    erp_scaler = joblib.load(os.path.join('features_extractors', f'MO_ERP_scaler_subject_{n}.pkl'))
    combined_clf = joblib.load(os.path.join('classifiers', f'MO_Combined_clf_subject_{n}.pkl'))

    # Create aligned epochs
    epochs_online_MO_FBCSP = create_online_epochs(online_file, 'MO', None, FBCSP_channels, [-0.5, 5.0], None, clean_data=False)
    epochs_online_MO_ERP = create_online_epochs(online_file, 'MO', None, ERP_channels, [-0.5, 5.0], [-0.5, 0], clean_data=False)

    MO_FBCSP_online_aligned, MO_ERP_online_aligned, labels_online_MO = align_epochs_and_labels(
        epochs_online_MO_FBCSP, epochs_online_MO_ERP, 'MO_Arm', 'MO_Leg'
    )

    # Run the whole-data classification method
    whole_data_results = online_whole_data_classification(
        MO_FBCSP_online_aligned, MO_ERP_online_aligned, labels_online_MO,
        fbcsp_clf, fbcsp_csp, erp_clf, combined_clf, MO_results['FBCSP'][subject]['selected_features'],
        MO_results['ERP'][subject]['selected_features'],
        fbcsp_scaler, erp_scaler
    )
    whole_epochs_results_MO[subject] = whole_data_results
    print(f"Whole-data classification results for Subject {subject}:")
    print(whole_data_results)

Online file found: cleaned_dataset\1\1_cleaned_online.set
Whole-data classification results for Subject 1:
{'fbcsp_accuracy': 0.5, 'erp_accuracy': 0.5555555555555556, 'combined_accuracy': 0.5}
Online file not found: cleaned_dataset\2\2_cleaned_online.set
Online file found: cleaned_dataset\3\3_cleaned_online.set
Whole-data classification results for Subject 3:
{'fbcsp_accuracy': 0.55, 'erp_accuracy': 0.55, 'combined_accuracy': 0.4}
Online file found: cleaned_dataset\4\4_cleaned_online.set
Whole-data classification results for Subject 4:
{'fbcsp_accuracy': 0.5, 'erp_accuracy': 0.45, 'combined_accuracy': 0.5}
Online file found: cleaned_dataset\5\5_cleaned_online.set
Whole-data classification results for Subject 5:
{'fbcsp_accuracy': 0.5294117647058824, 'erp_accuracy': 0.6470588235294118, 'combined_accuracy': 0.5294117647058824}
Online file found: cleaned_dataset\6\6_cleaned_online.set
Whole-data classification results for Subject 6:
{'fbcsp_accuracy': 0.5, 'erp_accuracy': 0.55, 'combined_

In [43]:
# Create a DataFrame to organize the data
data = []
for subject_id, scores in whole_epochs_results_AW.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'AW',
        'Accuracy': scores['combined_accuracy']
    })
for subject_id, scores in whole_epochs_results_MO.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'MO',
        'Accuracy': scores['combined_accuracy']
    })
for subject_id, scores in whole_epochs_results_MI.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'MI',
        'Accuracy': scores['combined_accuracy']
    })

df = pd.DataFrame(data)
df_grouped = df.iloc[:,1:].groupby('Condition').agg(['mean', 'std'])
print(df_grouped)

# Calculate means and stds for error bars
means = df.groupby('Condition')['Accuracy'].mean()
stds = df.groupby('Condition')['Accuracy'].std()

# Create Plotly grouped bar chart with error bars
fig = go.Figure()

for condition in df['Condition'].unique():
    cond_df = df[df['Condition'] == condition]
    fig.add_trace(go.Bar(
        x=cond_df['Subject'],
        y=cond_df['Accuracy'],
        name=condition,
        marker_color=colors[condition],
        error_y=dict(
            type='data',
            array=[stds[condition]] * len(cond_df),  # same std for all bars in this condition
            visible=True
        )
    ))

# Add chance level line
fig.add_hline(y=0.5, line_width=1, line_color="gray")

# Update layout to show subject IDs on x-axis ticks
fig.update_layout(
    barmode='group',
    xaxis_title='Subject',
    yaxis_title='Accuracy',
    title='Accuracy per Subject and Condition  - combined_accuracy',
    legend_title_text='Condition',
    xaxis=dict(
        tickmode='array',
        tickvals=df['Subject'].unique(),
        ticktext=df['Subject'].unique()
    ),
    yaxis=dict(range=[0.3, 1.0], tick0=0.4, dtick=0.1),
)

fig.show()


           Accuracy          
               mean       std
Condition                    
AW         0.480482  0.070290
MI         0.489304  0.072082
MO         0.509122  0.070818


In [44]:
# Create a DataFrame to organize the data
data = []
for subject_id, scores in whole_epochs_results_AW.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'AW',
        'Accuracy': scores['erp_accuracy']
    })
for subject_id, scores in whole_epochs_results_MO.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'MO',
        'Accuracy': scores['erp_accuracy']
    })
for subject_id, scores in whole_epochs_results_MI.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'MI',
        'Accuracy': scores['erp_accuracy']
    })

df = pd.DataFrame(data)
df_grouped = df.iloc[:,1:].groupby('Condition').agg(['mean', 'std'])
print(df_grouped)

# Calculate means and stds for error bars
means = df.groupby('Condition')['Accuracy'].mean()
stds = df.groupby('Condition')['Accuracy'].std()

# Create Plotly grouped bar chart with error bars
fig = go.Figure()

for condition in df['Condition'].unique():
    cond_df = df[df['Condition'] == condition]
    fig.add_trace(go.Bar(
        x=cond_df['Subject'],
        y=cond_df['Accuracy'],
        name=condition,
        marker_color=colors[condition],
        error_y=dict(
            type='data',
            array=[stds[condition]] * len(cond_df),  # same std for all bars in this condition
            visible=True
        )
    ))

# Add chance level line
fig.add_hline(y=0.5, line_width=1, line_color="gray")

# Update layout to show subject IDs on x-axis ticks
fig.update_layout(
    barmode='group',
    xaxis_title='Subject',
    yaxis_title='Accuracy',
    title='Accuracy per Subject and Condition  - erp_accuracy',
    legend_title_text='Condition',
    xaxis=dict(
        tickmode='array',
        tickvals=df['Subject'].unique(),
        ticktext=df['Subject'].unique()
    ),
    yaxis=dict(range=[0.3, 1.0], tick0=0.4, dtick=0.1),
)

fig.show()


           Accuracy          
               mean       std
Condition                    
AW         0.489130  0.108084
MI         0.448571  0.104354
MO         0.527405  0.106560


In [45]:
# Create a DataFrame to organize the data
data = []
for subject_id, scores in whole_epochs_results_AW.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'AW',
        'Accuracy': scores['fbcsp_accuracy']
    })
for subject_id, scores in whole_epochs_results_MO.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'MO',
        'Accuracy': scores['fbcsp_accuracy']
    })
for subject_id, scores in whole_epochs_results_MI.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'MI',
        'Accuracy': scores['fbcsp_accuracy']
    })

df = pd.DataFrame(data)
df_grouped = df.iloc[:,1:].groupby('Condition').agg(['mean', 'std'])
print(df_grouped)

# Calculate means and stds for error bars
means = df.groupby('Condition')['Accuracy'].mean()
stds = df.groupby('Condition')['Accuracy'].std()

# Create Plotly grouped bar chart with error bars
fig = go.Figure()

for condition in df['Condition'].unique():
    cond_df = df[df['Condition'] == condition]
    fig.add_trace(go.Bar(
        x=cond_df['Subject'],
        y=cond_df['Accuracy'],
        name=condition,
        marker_color=colors[condition],
        error_y=dict(
            type='data',
            array=[stds[condition]] * len(cond_df),  # same std for all bars in this condition
            visible=True
        )
    ))

# Add chance level line
fig.add_hline(y=0.5, line_width=1, line_color="gray")

# Update layout to show subject IDs on x-axis ticks
fig.update_layout(
    barmode='group',
    xaxis_title='Subject',
    yaxis_title='Accuracy',
    title='Accuracy per Subject and Condition  - fbcsp_accuracy',
    legend_title_text='Condition',
    xaxis=dict(
        tickmode='array',
        tickvals=df['Subject'].unique(),
        ticktext=df['Subject'].unique()
    ),
    yaxis=dict(range=[0.3, 1.0], tick0=0.4, dtick=0.1),
)

fig.show()


           Accuracy          
               mean       std
Condition                    
AW         0.480391  0.073169
MI         0.475458  0.083770
MO         0.503850  0.050450


In [46]:

all_online_AW_results_seen = {}
all_online_AW_results_new = {}
for n, subject in enumerate(subjects):
    online_file = os.path.join(data_folder, str(subject), f'{subject}_cleaned_online.set')
    if os.path.exists(online_file):
        print(f"Online file found: {online_file}")
    else:
        print(f"Online file not found: {online_file}")
        continue
    all_online_AW_results_seen[subject] = {}
    all_online_AW_results_new[subject] = {}

    AW_FBCSP_csp= joblib.load(os.path.join('features_extractors', 'AW_FBCSP_csp_subject_'+str(n)+'.pkl'))
    AW_FBCSP_scaler = joblib.load(os.path.join('features_extractors', 'AW_FBCSP_csp_scaler_subject_'+str(n)+'.pkl'))
    AW_FBCSP_clf = joblib.load(os.path.join('classifiers', 'AW_FBCSP_clf_subject_'+str(n)+'.pkl'))
    AW_ERP_clf = joblib.load(os.path.join('classifiers', 'AW_ERP_clf_subject_'+str(n)+'.pkl'))
    AW_ERP_scaler = joblib.load(os.path.join('features_extractors', 'AW_ERP_scaler_subject_'+str(n)+'.pkl'))
    AW_combined_clf = joblib.load(os.path.join('classifiers', 'AW_Combined_clf_subject_'+str(n)+'.pkl'))

    epochs_online_AW_FBCSP = create_online_epochs(online_file, 'AW','New_labels', FBCSP_channels, [-0.5, 5.0], None, clean_data=False)
    seen_epochs_FBCSP = epochs_online_AW_FBCSP['AW_Arm', 'AW_Leg']
    new_epochs_FBCSP = epochs_online_AW_FBCSP['AW_APLAUDIR', 'AW_PATEAR']

    epochs_online_AW_ERP = create_online_epochs(online_file, 'AW', 'New_labels', ERP_channels, [-0.5, 5.0], [-0.5, 0], clean_data=False)
    seen_epochs_ERP = epochs_online_AW_ERP['AW_Arm', 'AW_Leg']
    new_epochs_ERP = epochs_online_AW_ERP['AW_APLAUDIR', 'AW_PATEAR']

    # Align epochs and labels
    
    AW_FBCSP_online_aligned_seen, AW_ERP_online_aligned_seen, labels_online_AW_seen = align_epochs_and_labels(seen_epochs_FBCSP, seen_epochs_ERP,'AW_Arm', 'AW_Leg')
    AW_FBCSP_online_aligned_new, AW_ERP_online_aligned_new, labels_online_AW_new = align_epochs_and_labels(new_epochs_FBCSP, new_epochs_ERP,'AW_APLAUDIR', 'AW_PATEAR')

    models_and_selectors = {'fbcsp_clf':AW_FBCSP_clf,'fbcsp_csp': AW_FBCSP_csp, 'fbcsp_selected_features': AW_results['FBCSP'][subject]['selected_features'], 'fbcsp_scaler': AW_FBCSP_scaler,
                            'erp_clf': AW_ERP_clf, 'erp_selected_features': AW_results['ERP'][subject]['selected_features'], 'erp_scaler': AW_ERP_scaler,
                            'combined_clf': AW_combined_clf}
    
    results = online_overtime_classification_from_whole_epochs(AW_FBCSP_online_aligned_seen, AW_ERP_online_aligned_seen, labels_online_AW_seen, models_and_selectors) 
    all_online_AW_results_seen[subject] = results

    results = online_overtime_classification_from_whole_epochs(AW_FBCSP_online_aligned_new, AW_ERP_online_aligned_new, labels_online_AW_new, models_and_selectors)
    all_online_AW_results_new[subject] = results

Online file found: cleaned_dataset\1\1_cleaned_online.set
Online file not found: cleaned_dataset\2\2_cleaned_online.set
Online file found: cleaned_dataset\3\3_cleaned_online.set
Online file found: cleaned_dataset\4\4_cleaned_online.set
Online file found: cleaned_dataset\5\5_cleaned_online.set
Online file found: cleaned_dataset\6\6_cleaned_online.set
Online file found: cleaned_dataset\7\7_cleaned_online.set
Online file found: cleaned_dataset\8\8_cleaned_online.set
Online file found: cleaned_dataset\9\9_cleaned_online.set
Online file found: cleaned_dataset\10\10_cleaned_online.set
Online file found: cleaned_dataset\11\11_cleaned_online.set
Online file found: cleaned_dataset\12\12_cleaned_online.set
Online file found: cleaned_dataset\13\13_cleaned_online.set
Online file found: cleaned_dataset\14\14_cleaned_online.set
Online file found: cleaned_dataset\15\15_cleaned_online.set
Online file found: cleaned_dataset\16\16_cleaned_online.set
Online file found: cleaned_dataset\17\17_cleaned_onlin

In [47]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import warnings

# Suppress the UserWarning from Plotly
warnings.filterwarnings('ignore', category=UserWarning, module='plotly.graph_objects')

# Prepare data for plotting
plot_data = {}
for subject, results in all_online_AW_results_seen.items():
    if not results:
        continue
    
    # Calculate mean accuracy for each model across all time points
    fbcsp_mean = np.mean(results['fbcsp_accuracies'])
    erp_mean = np.mean(results['erp_accuracies'])
    combined_mean = np.mean(results['combined_accuracies'])
    
    # Find the best model based on mean accuracy
    means = {'FBCSP': fbcsp_mean, 'ERP': erp_mean, 'Combined': combined_mean}
    best_model_name = max(means, key=means.get)
    
    # Store the accuracies of the best model and the model name
    plot_data[subject] = {
        'model_name': best_model_name,
        'accuracies': results[f'{best_model_name.lower()}_accuracies']
    }

# Create a DataFrame for easy plotting
x_times = all_online_AW_results_seen['1']['times']
y_subjects = []
z_accuracies = []
custom_data = []

for subject, data in plot_data.items():
    y_subjects.append(f"Subject {subject}: {data['model_name']}")
    z_accuracies.append(data['accuracies'])

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    x=x_times,
    y=y_subjects,
    z=z_accuracies,
    colorscale=[[0.0, 'red'], [0.5, 'white'], [1.0, 'blue']],
    zmin=0,
    zmax=1,
    colorbar=dict(title="Accuracy")
))

# Set layout titles and labels
fig.update_layout(
    height=800,
    width=1200,
    title='Best Model Accuracy Over Time for Each Subject',
    xaxis_title='Time (s)',
    yaxis_title='Subject and Best Model',
    yaxis_autorange='reversed',
    xaxis_tickmode='linear',
    xaxis_dtick=0.5
)

fig.show()

In [48]:
# Calculate and print the average and standard deviation of the best model accuracy for each subject
subject_averages = {}
subject_stds = {}

for subject, data in plot_data.items():
    accuracies = np.array(data['accuracies'])
    subject_averages[subject] = np.mean(accuracies)
    subject_stds[subject] = np.std(accuracies)
    print(f"Subject {subject}: Best Model = {data['model_name']}, Mean Accuracy = {subject_averages[subject]:.3f}, Std = {subject_stds[subject]:.3f}")

Subject 1: Best Model = ERP, Mean Accuracy = 0.568, Std = 0.123
Subject 3: Best Model = ERP, Mean Accuracy = 0.523, Std = 0.129
Subject 4: Best Model = ERP, Mean Accuracy = 0.580, Std = 0.144
Subject 5: Best Model = FBCSP, Mean Accuracy = 0.623, Std = 0.126
Subject 6: Best Model = FBCSP, Mean Accuracy = 0.568, Std = 0.062
Subject 7: Best Model = FBCSP, Mean Accuracy = 0.455, Std = 0.102
Subject 8: Best Model = FBCSP, Mean Accuracy = 0.500, Std = 0.000
Subject 9: Best Model = FBCSP, Mean Accuracy = 0.571, Std = 0.000
Subject 10: Best Model = ERP, Mean Accuracy = 0.345, Std = 0.172
Subject 11: Best Model = Combined, Mean Accuracy = 0.523, Std = 0.158
Subject 12: Best Model = Combined, Mean Accuracy = 0.591, Std = 0.056
Subject 13: Best Model = FBCSP, Mean Accuracy = 0.500, Std = 0.000
Subject 14: Best Model = ERP, Mean Accuracy = 0.523, Std = 0.198
Subject 15: Best Model = Combined, Mean Accuracy = 0.481, Std = 0.140
Subject 16: Best Model = ERP, Mean Accuracy = 0.519, Std = 0.126
Subjec

In [49]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import warnings

# Suppress the UserWarning from Plotly
warnings.filterwarnings('ignore', category=UserWarning, module='plotly.graph_objects')

# Prepare data for plotting
plot_data = {}
for subject, results in all_online_AW_results_new.items():
    if not results:
        continue
    
    # Calculate mean accuracy for each model across all time points
    fbcsp_mean = np.mean(results['fbcsp_accuracies'])
    erp_mean = np.mean(results['erp_accuracies'])
    combined_mean = np.mean(results['combined_accuracies'])
    
    # Find the best model based on mean accuracy
    means = {'FBCSP': fbcsp_mean, 'ERP': erp_mean, 'Combined': combined_mean}
    best_model_name = max(means, key=means.get)
    
    # Store the accuracies of the best model and the model name
    plot_data[subject] = {
        'model_name': best_model_name,
        'accuracies': results[f'{best_model_name.lower()}_accuracies']
    }

# Create a DataFrame for easy plotting
x_times = all_online_AW_results_new['1']['times']
y_subjects = []
z_accuracies = []
custom_data = []

for subject, data in plot_data.items():
    y_subjects.append(f"Subject {subject}: {data['model_name']}")
    z_accuracies.append(data['accuracies'])

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    x=x_times,
    y=y_subjects,
    z=z_accuracies,
    colorscale=[[0.0, 'red'], [0.5, 'white'], [1.0, 'blue']],
    zmin=0,
    zmax=1,
    colorbar=dict(title="Accuracy")
))

# Set layout titles and labels
fig.update_layout(
    height=800,
    width=1200,
    title='Best Model Accuracy Over Time for Each Subject',
    xaxis_title='Time (s)',
    yaxis_title='Subject and Best Model',
    yaxis_autorange='reversed',
    xaxis_tickmode='linear',
    xaxis_dtick=0.5
)

fig.show()



In [50]:
# Calculate and print the average and standard deviation of the best model accuracy for each subject
subject_averages = {}
subject_stds = {}

for subject, data in plot_data.items():
    accuracies = np.array(data['accuracies'])
    subject_averages[subject] = np.mean(accuracies)
    subject_stds[subject] = np.std(accuracies)
    print(f"Subject {subject}: Best Model = {data['model_name']}, Mean Accuracy = {subject_averages[subject]:.3f}, Std = {subject_stds[subject]:.3f}")

Subject 1: Best Model = ERP, Mean Accuracy = 0.568, Std = 0.069
Subject 3: Best Model = FBCSP, Mean Accuracy = 0.500, Std = 0.000
Subject 4: Best Model = FBCSP, Mean Accuracy = 0.500, Std = 0.000
Subject 5: Best Model = FBCSP, Mean Accuracy = 0.496, Std = 0.045
Subject 6: Best Model = FBCSP, Mean Accuracy = 0.587, Std = 0.060
Subject 7: Best Model = FBCSP, Mean Accuracy = 0.595, Std = 0.090
Subject 8: Best Model = FBCSP, Mean Accuracy = 0.455, Std = 0.000
Subject 9: Best Model = ERP, Mean Accuracy = 0.612, Std = 0.096
Subject 10: Best Model = Combined, Mean Accuracy = 0.444, Std = 0.000
Subject 11: Best Model = FBCSP, Mean Accuracy = 0.492, Std = 0.075
Subject 12: Best Model = ERP, Mean Accuracy = 0.705, Std = 0.120
Subject 13: Best Model = ERP, Mean Accuracy = 0.629, Std = 0.109
Subject 14: Best Model = FBCSP, Mean Accuracy = 0.614, Std = 0.129
Subject 15: Best Model = ERP, Mean Accuracy = 0.512, Std = 0.124
Subject 16: Best Model = FBCSP, Mean Accuracy = 0.535, Std = 0.093
Subject 17

In [51]:
whole_epochs_results_AW_seen = {}
whole_epochs_results_AW_new = {}
for n, subject in enumerate(subjects):
    online_file = os.path.join(data_folder, str(subject), f'{subject}_cleaned_online.set')
    if os.path.exists(online_file):
        print(f"Online file found: {online_file}")
    else:
        print(f"Online file not found: {online_file}")
        continue
    whole_epochs_results_AW_seen[subject] = {}
    whole_epochs_results_AW_new[subject] = {}

    # Load pre-trained models
    fbcsp_clf = joblib.load(os.path.join('classifiers', f'AW_FBCSP_clf_subject_{n}.pkl'))
    fbcsp_csp = joblib.load(os.path.join('features_extractors', f'AW_FBCSP_csp_subject_{n}.pkl'))
    fbcsp_scaler = joblib.load(os.path.join('features_extractors', f'AW_FBCSP_csp_scaler_subject_{n}.pkl'))
    erp_clf = joblib.load(os.path.join('classifiers', f'AW_ERP_clf_subject_{n}.pkl'))
    erp_scaler = joblib.load(os.path.join('features_extractors', f'AW_ERP_scaler_subject_{n}.pkl'))
    combined_clf = joblib.load(os.path.join('classifiers', f'AW_Combined_clf_subject_{n}.pkl'))

    # Create epochs with new labels
    epochs_online_AW_FBCSP = create_online_epochs(online_file, 'AW', 'New_labels', FBCSP_channels, [-0.5, 5.0], None, clean_data=False)
    seen_epochs_FBCSP = epochs_online_AW_FBCSP['AW_Arm', 'AW_Leg']
    new_epochs_FBCSP = epochs_online_AW_FBCSP['AW_APLAUDIR', 'AW_PATEAR']

    epochs_online_AW_ERP = create_online_epochs(online_file, 'AW', 'New_labels', ERP_channels, [-0.5, 5.0], [-0.5, 0], clean_data=False)
    seen_epochs_ERP = epochs_online_AW_ERP['AW_Arm', 'AW_Leg']
    new_epochs_ERP = epochs_online_AW_ERP['AW_APLAUDIR', 'AW_PATEAR']

    # Align epochs and labels
    AW_FBCSP_online_aligned_seen, AW_ERP_online_aligned_seen, labels_online_AW_seen = align_epochs_and_labels(seen_epochs_FBCSP, seen_epochs_ERP, 'AW_Arm', 'AW_Leg')
    AW_FBCSP_online_aligned_new, AW_ERP_online_aligned_new, labels_online_AW_new = align_epochs_and_labels(new_epochs_FBCSP, new_epochs_ERP, 'AW_APLAUDIR', 'AW_PATEAR')

    # Whole-data classification for seen labels
    results_seen = online_whole_data_classification(
        AW_FBCSP_online_aligned_seen, AW_ERP_online_aligned_seen, labels_online_AW_seen,
        fbcsp_clf, fbcsp_csp, erp_clf, combined_clf,
        AW_results['FBCSP'][subject]['selected_features'], AW_results['ERP'][subject]['selected_features'],
        fbcsp_scaler, erp_scaler
    )
    whole_epochs_results_AW_seen[subject] = results_seen
    print(f"Whole-data classification results for Subject {subject} (seen):")
    print(results_seen)

    # Whole-data classification for new labels
    results_new = online_whole_data_classification(
        AW_FBCSP_online_aligned_new, AW_ERP_online_aligned_new, labels_online_AW_new,
        fbcsp_clf, fbcsp_csp, erp_clf, combined_clf,
        AW_results['FBCSP'][subject]['selected_features'], AW_results['ERP'][subject]['selected_features'],
        fbcsp_scaler, erp_scaler
    )
    whole_epochs_results_AW_new[subject] = results_new
    print(f"Whole-data classification results for Subject {subject} (new):")
    print(results_new)

Online file found: cleaned_dataset\1\1_cleaned_online.set
Whole-data classification results for Subject 1 (seen):
{'fbcsp_accuracy': 0.625, 'erp_accuracy': 0.625, 'combined_accuracy': 0.5}
Whole-data classification results for Subject 1 (new):
{'fbcsp_accuracy': 0.4166666666666667, 'erp_accuracy': 0.6666666666666666, 'combined_accuracy': 0.5}
Online file not found: cleaned_dataset\2\2_cleaned_online.set
Online file found: cleaned_dataset\3\3_cleaned_online.set
Whole-data classification results for Subject 3 (seen):
{'fbcsp_accuracy': 0.5, 'erp_accuracy': 0.5, 'combined_accuracy': 0.5}
Whole-data classification results for Subject 3 (new):
{'fbcsp_accuracy': 0.5, 'erp_accuracy': 0.5, 'combined_accuracy': 0.5}
Online file found: cleaned_dataset\4\4_cleaned_online.set
Whole-data classification results for Subject 4 (seen):
{'fbcsp_accuracy': 0.5, 'erp_accuracy': 0.5, 'combined_accuracy': 0.5}
Whole-data classification results for Subject 4 (new):
{'fbcsp_accuracy': 0.5, 'erp_accuracy': 0.

In [52]:
# Create a DataFrame to organize the data
data = []
for subject_id, scores in whole_epochs_results_AW_new.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'AW_2',
        'Accuracy': scores['combined_accuracy']
    })
for subject_id, scores in whole_epochs_results_AW_seen.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'AW',
        'Accuracy': scores['combined_accuracy']
    })


df = pd.DataFrame(data)
df_grouped = df.iloc[:,1:].groupby('Condition').agg(['mean', 'std'])
print(df_grouped)

# Calculate means and stds for error bars
means = df.groupby('Condition')['Accuracy'].mean()
stds = df.groupby('Condition')['Accuracy'].std()

# Create Plotly grouped bar chart with error bars
fig = go.Figure()

for condition in df['Condition'].unique():
    cond_df = df[df['Condition'] == condition]
    fig.add_trace(go.Bar(
        x=cond_df['Subject'],
        y=cond_df['Accuracy'],
        name=condition,
        marker_color=colors[condition],
        error_y=dict(
            type='data',
            array=[stds[condition]] * len(cond_df),  # same std for all bars in this condition
            visible=True
        )
    ))

# Add chance level line
fig.add_hline(y=0.5, line_width=1, line_color="gray")

# Update layout to show subject IDs on x-axis ticks
fig.update_layout(
    barmode='group',
    xaxis_title='Subject',
    yaxis_title='Accuracy',
    title='Accuracy per Subject and Condition  - combined_accuracy',
    legend_title_text='Condition',
    xaxis=dict(
        tickmode='array',
        tickvals=df['Subject'].unique(),
        ticktext=df['Subject'].unique()
    ),
    yaxis=dict(range=[0.3, 1.0], tick0=0.4, dtick=0.1),
)

fig.show()


           Accuracy          
               mean       std
Condition                    
AW         0.441857  0.129262
AW_2       0.505071  0.074969


In [53]:
# Create a DataFrame to organize the data
data = []
for subject_id, scores in whole_epochs_results_AW_new.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'AW_2',
        'Accuracy': scores['fbcsp_accuracy']
    })
for subject_id, scores in whole_epochs_results_AW_seen.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'AW',
        'Accuracy': scores['fbcsp_accuracy']
    })


df = pd.DataFrame(data)
df_grouped = df.iloc[:,1:].groupby('Condition').agg(['mean', 'std'])
print(df_grouped)

# Calculate means and stds for error bars
means = df.groupby('Condition')['Accuracy'].mean()
stds = df.groupby('Condition')['Accuracy'].std()

# Create Plotly grouped bar chart with error bars
fig = go.Figure()

for condition in df['Condition'].unique():
    cond_df = df[df['Condition'] == condition]
    fig.add_trace(go.Bar(
        x=cond_df['Subject'],
        y=cond_df['Accuracy'],
        name=condition,
        marker_color=colors[condition],
        error_y=dict(
            type='data',
            array=[stds[condition]] * len(cond_df),  # same std for all bars in this condition
            visible=True
        )
    ))

# Add chance level line
fig.add_hline(y=0.5, line_width=1, line_color="gray")

# Update layout to show subject IDs on x-axis ticks
fig.update_layout(
    barmode='group',
    xaxis_title='Subject',
    yaxis_title='Accuracy',
    title='Accuracy per Subject and Condition  - combined_accuracy',
    legend_title_text='Condition',
    xaxis=dict(
        tickmode='array',
        tickvals=df['Subject'].unique(),
        ticktext=df['Subject'].unique()
    ),
    yaxis=dict(range=[0.3, 1.0], tick0=0.4, dtick=0.1),
)

fig.show()


           Accuracy          
               mean       std
Condition                    
AW         0.438857  0.131999
AW_2       0.504929  0.086007


In [54]:
# Create a DataFrame to organize the data
data = []
for subject_id, scores in whole_epochs_results_AW_new.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'AW_2',
        'Accuracy': scores['erp_accuracy']
    })
for subject_id, scores in whole_epochs_results_AW_seen.items():
    data.append({
        'Subject': subject_id,
        'Condition': 'AW',
        'Accuracy': scores['erp_accuracy']
    })


df = pd.DataFrame(data)
df_grouped = df.iloc[:,1:].groupby('Condition').agg(['mean', 'std'])
print(df_grouped)

# Calculate means and stds for error bars
means = df.groupby('Condition')['Accuracy'].mean()
stds = df.groupby('Condition')['Accuracy'].std()

# Create Plotly grouped bar chart with error bars
fig = go.Figure()

for condition in df['Condition'].unique():
    cond_df = df[df['Condition'] == condition]
    fig.add_trace(go.Bar(
        x=cond_df['Subject'],
        y=cond_df['Accuracy'],
        name=condition,
        marker_color=colors[condition],
        error_y=dict(
            type='data',
            array=[stds[condition]] * len(cond_df),  # same std for all bars in this condition
            visible=True
        )
    ))

# Add chance level line
fig.add_hline(y=0.5, line_width=1, line_color="gray")

# Update layout to show subject IDs on x-axis ticks
fig.update_layout(
    barmode='group',
    xaxis_title='Subject',
    yaxis_title='Accuracy',
    title='Accuracy per Subject and Condition  - combined_accuracy',
    legend_title_text='Condition',
    xaxis=dict(
        tickmode='array',
        tickvals=df['Subject'].unique(),
        ticktext=df['Subject'].unique()
    ),
    yaxis=dict(range=[0.0, 1.0], tick0=0.4, dtick=0.1),
)

fig.show()


           Accuracy          
               mean       std
Condition                    
AW         0.473286  0.187381
AW_2       0.497838  0.140330
